# Platogram: Quick Start

## Introduction
Using this Notebook you can transform a an audio from YouTube or any source (Google Drive, Fathom, etc.) into paragraphs that spark joy. The resulting text is intended to be a lossless readable representation of the original audio, and will contain references to original transcript of the audio. You will also get summary and title.

Additionally you can store the results in the library and prompt it to generate new content with references to the original audio and transcript, or query it to retrieve paragraphs and underlying transcript.

We used the text of Chapter 1: Atoms in Motion from Volume 1. of Feynman's Lectures on Physics and a corresponding transcript obtained from https://www.feynmanlectures.caltech.edu/ as an example and reached out to Cal Tech for clarifications about fair use of the material. We observed high-quality output on single-speaker as well as multi-speaker audio in domains of eduction, family law, and business.

## Request for Feedback
We are looking for your feedback on the quality of the output for your use case and your suggestions on how to improve Platogram. Please reach out to us at platogram@codeanyway.com or ping Artyom in [Telegram](https://t.me/astaff239).

## Setup
To use this notebook you need:
1. Access to [Anthropic Claude](https://www.anthropic.com/claude) and API key ([create key](https://console.anthropic.com/settings/keys)).
2. (Optional) If subtitles are not available, you will need access to [Assembly AI](https://www.assemblyai.com/dashboard/signup) and API ([create key](https://www.assemblyai.com/app/)).

Ping on [Telegram](https://t.me/astaff239) or Discord.

## Costs
* Indexing is about 0.75 USD per hour of audio using Anthropic Claude 3.5 Sonnet model.
* Transcription is about 0.37 USD per hour of audio using Assembly AI Best model.

## Usage
* Update `URL`, `ANTHROPIC_API_KEY` and optionally `ASSEMBLYAI_API_KEY` below.
* Run the notebook by clicking "Run All" button above.

In [1]:
#@title Install Platogram
%%capture
!pip install git+https://github.com/code-anyway/platogram.git
!sudo apt-get update && sudo apt-get install -y ffmpeg

# API Keys

* Set `ANTHROPIC_API_KEY` to your Anthropic API key: https://console.anthropic.com/settings/keys
* (Optional) Set `ASSEMBLYAI_API_KEY` to your Assembly AI API key: https://www.assemblyai.com/app/


In [1]:
ANTHROPIC_API_KEY="sk-ant-api..."
ASSEMBLYAI_API_KEY=None

# URL

* You can also use Dropbox, Fathom, etc as sources of audio.
* Set the `URL` to `"file:///path/to/file.mp3"` for local files.
* **Remember**: you can mount your Google Drive from Colab from File Browser and then use something like `"file:///content/drive/MyDrive/memos/01.mp3"`.

In [2]:
URL = "https://youtu.be/NvLlpY9vd9E"

In [3]:
#@title Import Platogram and Initialize LLM and ASR Models
import platogram as plato


# Indexing is about $0.75 per hour of audio using Anthropic Claude 3.5 Sonnet model
llm = plato.llm.get_model("anthropic/claude-3-5-sonnet", ANTHROPIC_API_KEY)

# Transcription is about $0.37 per hour of audio using Assembly AI Best model
asr = plato.asr.get_model("assembly-ai/best", ASSEMBLYAI_API_KEY) if ASSEMBLYAI_API_KEY else None

In [4]:
#@title Extract Transcript and create Index at about 1 minute per 12 minutes of audio.

transcript = plato.extract_transcript(URL, asr)
content = plato.index(transcript, llm)

[youtube] Extracting URL: https://youtu.be/NvLlpY9vd9E
[youtube] NvLlpY9vd9E: Downloading webpage
[youtube] NvLlpY9vd9E: Downloading ios player API JSON
[youtube] NvLlpY9vd9E: Downloading m3u8 information
[youtube] Extracting URL: https://youtu.be/NvLlpY9vd9E
[youtube] NvLlpY9vd9E: Downloading webpage
[youtube] NvLlpY9vd9E: Downloading ios player API JSON
[youtube] NvLlpY9vd9E: Downloading player 5b22937f
[youtube] NvLlpY9vd9E: Downloading m3u8 information
[info] NvLlpY9vd9E: Downloading 1 format(s): 251
[download] Destination: /tmp/tmp_rzbrg3s/NvLlpY9vd9E.webm
[download] 100% of   33.93MiB in 00:00:00 at 38.07MiB/s    


100%|██████████| 6/6 [02:38<00:00, 26.50s/it]


In [6]:
#@title Render Paragraphs and Transcript into Markdown

from IPython.display import display, Markdown
import re

from platogram.utils import format_time


def render_reference(i: int, first: int) -> str:
    link = f" [[{i-first+1}]]({URL}#t={content.transcript[i].time_ms // 1000})"
    return link


def render_transcript(first: int, last: int) -> str:
    return "<br>".join([
        f"{i+1}. <a href='{URL}#t={t.time_ms // 1000}'>{format_time(t.time_ms)}</a>: {t.text}"
        for i, t in enumerate(content.transcript[first:last+1])
    ])


def render_paragraph(p: str) -> str:
    references = sorted([int(i) for i in re.findall(r"【(\d+)】", p)])
    if not references:
        return p

    first = references[0]
    last = references[-1]

    paragraph = re.sub(
        r"【(\d+)】",
        lambda match: render_reference(int(match.group(1)), first),
        p
    )
    transcript = render_transcript(first, last)
    return f"{paragraph}\n<details><summary>Transcript</summary>{transcript}</details>"

In [7]:
#@title Render Content

markdown = "\n\n".join([
    render_paragraph(p)
    for p in content.passages
])

markdown = f"""
# {content.title}
{content.short_summary}
## Summary
{content.summary}
## Content
{markdown}
"""
display(Markdown(markdown))


# The Illusion of Self: Rethinking Our Place in the Universe
This text challenges our perception of self, arguing that we are not separate entities but part of an interconnected universe. It suggests that realizing this interconnectedness is crucial for addressing environmental issues and using technology responsibly. The speaker proposes practical steps to implement this new understanding.
## Summary
This text explores the concept of self and our relationship with the universe, challenging traditional notions of individuality and separateness. The speaker argues that our common understanding of "I" as a separate entity is a misconception, proposing instead that we are part of a unified, interconnected process with our environment. They discuss how this misunderstanding has led to a hostile approach to nature and technology, causing environmental damage. The talk covers various philosophical and scientific perspectives, including ideas from Alfred North Whitehead, Hindu concepts of Maya, and modern physics. The speaker emphasizes the need for a transformation in human consciousness to better align with this interconnected reality, suggesting that such a shift is crucial for responsible use of technology and addressing environmental issues. The discussion concludes by proposing practical steps, such as establishing institutes for ecological technology, to implement this new understanding in real-world scenarios.
## Content
We talk in ordinary ways about "my body," "my feet." When we go to the dentist to have [[1]](https://youtu.be/NvLlpY9vd9E#t=97) our teeth fixed, we regard him rather as a mechanic. Just as you take your car to the garage, [[2]](https://youtu.be/NvLlpY9vd9E#t=106) you take your body to the surgeon or the dentist to be fixed, [[3]](https://youtu.be/NvLlpY9vd9E#t=114) to have parts changed or something of that kind. They're really getting to work on that [[4]](https://youtu.be/NvLlpY9vd9E#t=118) now. So the question arises: when somebody has a heart transplant, that sounds very radical [[5]](https://youtu.be/NvLlpY9vd9E#t=124) because we say "in my heart of hearts." But nowadays, most of us seem to feel that whatever [[6]](https://youtu.be/NvLlpY9vd9E#t=132) "I" is, it is located in the head. Somewhere behind the eyes and between the [[7]](https://youtu.be/NvLlpY9vd9E#t=141) ears is the center, and the rest of us is an appendage, a vehicle which carries the self [[8]](https://youtu.be/NvLlpY9vd9E#t=150) around.
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=97'>01:37</a>: We talk in ordinary ways about my body. My feet. And when we go to the dentist to have<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=106'>01:46</a>: out teeth fixed, we regard him rather as a mechanic. Like you take your car to the garage<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=114'>01:54</a>: so you take your body to the surgeon or the dentist or whatever it is to be fixed how<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=118'>01:58</a>: the parts changed or something of that kind. And they're really getting to work on that<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=124'>02:04</a>: now. And so the question is when somebody has a heart transplant. That sounds very radical<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=132'>02:12</a>: because we say in my heart of hearts. But nowadays most of us seem to feel that whatever<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=141'>02:21</a>: it is, that I is, is located in the head. Somewhere behind the eyes and between the<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=150'>02:30</a>: ears is the center. And the rest of us is an appendage a vehicle which carries the self</details>

Popular speech also reflects the sensation that "I" am very different from what [[1]](https://youtu.be/NvLlpY9vd9E#t=157) we call "the other." Other people, other things - anything that we can become aware of is [[2]](https://youtu.be/NvLlpY9vd9E#t=171) sort of "other." There is an apparent opposition between the knower and the known. We [[3]](https://youtu.be/NvLlpY9vd9E#t=179) talk about "facing reality" or "coming into this world," as if somehow we didn't belong, [[4]](https://youtu.be/NvLlpY9vd9E#t=186) as if instead of being leaves growing out of a tree, we were a lot of birds that had [[5]](https://youtu.be/NvLlpY9vd9E#t=196) alighted on bare branches. It has become common sense for most people living in the [[6]](https://youtu.be/NvLlpY9vd9E#t=201) twentieth century today to adopt the nineteenth-century philosophy of science, which interprets [[7]](https://youtu.be/NvLlpY9vd9E#t=210) the physical universe outside human bodies as being a mechanical contraption which is [[8]](https://youtu.be/NvLlpY9vd9E#t=220) essentially stupid, unfeeling, automatic, composed of mainly geological elements: rocks, gases, [[9]](https://youtu.be/NvLlpY9vd9E#t=229) and so forth. Therefore, we feel rather alone and left out of this thing. In contrast [[10]](https://youtu.be/NvLlpY9vd9E#t=240) with the ideas of Ptolemaic astronomy, instead of being at the center of the universe, we [[11]](https://youtu.be/NvLlpY9vd9E#t=248) are on the outer limits of a minor galaxy revolving around an unimportant star, on a [[12]](https://youtu.be/NvLlpY9vd9E#t=257) small, minute ball of rock. That astronomical way of looking at things is simply [[13]](https://youtu.be/NvLlpY9vd9E#t=265) overwhelming. It makes us feel not only of no importance but also very much left out. [[14]](https://youtu.be/NvLlpY9vd9E#t=275) As a result, that is the common sense of most people living today. [[15]](https://youtu.be/NvLlpY9vd9E#t=284)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=157'>02:37</a>: around. Now popular speech also reflects the sensation. That I am very different from what<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=171'>02:51</a>: we call the other. Other people, other things. Anything that we can become aware of is sort<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=179'>02:59</a>: of other. There is an opposition apparently between the knower and the known. And so we<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=186'>03:06</a>: talk about facing reality. We talk about coming into this world. As if somehow we didn't belong.<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=196'>03:16</a>: As if instead of being leaves growing out of a tree we were a lot of birds that had<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=201'>03:21</a>: alighted on bare branches. And it has become common sense for most people living in the<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=210'>03:30</a>: twentieth century today to adopt the nineteenth century philosophy of science. Which interprets<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=220'>03:40</a>: the physical universe outside human bodies as being a mechanical contraption which is<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=229'>03:49</a>: essentially stupid, unfeeling automatic, Composed of mainly geological elements: rocks gases<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=240'>04:00</a>: and so forth. And therefore we feel rather alone and left out of this thing. In contrast<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=248'>04:08</a>: with the ideas of Ptolemaic astronomy. Instead of being at the center of the universe. We<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=257'>04:17</a>: are on the outer limits of a minor galaxy revolving around an unimportant star. On a<br>13. <a href='https://youtu.be/NvLlpY9vd9E#t=265'>04:25</a>: small minute ball of rock. And therefore that astronomical way of looking at things is simply<br>14. <a href='https://youtu.be/NvLlpY9vd9E#t=275'>04:35</a>: overwhelming. It makes us feel not only of no importance but also very much left out.<br>15. <a href='https://youtu.be/NvLlpY9vd9E#t=284'>04:44</a>: And as a result, that is the common sense of most people living today.</details>

We did, of course, have a religious view of our nature: that we were the children of a [[1]](https://youtu.be/NvLlpY9vd9E#t=290) loving God who is in charge of this whole operation. But very few people actually believe [[2]](https://youtu.be/NvLlpY9vd9E#t=294) that anymore. A great many people think they ought to believe it and would like to believe [[3]](https://youtu.be/NvLlpY9vd9E#t=301) it, but they don't. Most ministers that I know don't believe it, but they feel guilty [[4]](https://youtu.be/NvLlpY9vd9E#t=306) about this because they feel they ought to. It became implausible. There never was [[5]](https://youtu.be/NvLlpY9vd9E#t=311) a serious argument against it; it simply became unthinkable in comparison with the dimensions [[6]](https://youtu.be/NvLlpY9vd9E#t=317) of the universe as we now see it. So, having lost a way of looking at the world, an image [[7]](https://youtu.be/NvLlpY9vd9E#t=323) of the world which gave us some sense of meaning, we now have an image of the world which gives [[8]](https://youtu.be/NvLlpY9vd9E#t=332) us none at all. And so we feel rather inclined to put up a fight against the whole show. [[9]](https://youtu.be/NvLlpY9vd9E#t=337)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=290'>04:50</a>: We did of course have a religious view of our nature that we were the children of a<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=294'>04:54</a>: loving God who is in charge of this whole operation. But very few people actually believe<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=301'>05:01</a>: that anymore. A great many people think they ought to believe it and would like to believe<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=306'>05:06</a>: in it but they don't. Most ministers that I know don't believe it but they feel guilty<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=311'>05:11</a>: about this because they feel they ought to. But it became implausible. There never was<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=317'>05:17</a>: a serious argument against it. It simply became unthinkable in comparison with the dimensions<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=323'>05:23</a>: of the universe as we now see it. So having lost a way of looking at the world, an image<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=332'>05:32</a>: of the world which gave us some sense of meaning, we now have an image of the world which gives<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=337'>05:37</a>: us none at all. And so we feel rather inclined to put up a fight against the whole show.</details>

Interestingly enough, when in the nineteenth century we switched our common sense from [[1]](https://youtu.be/NvLlpY9vd9E#t=346) supernaturalism to naturalism, one would think that a naturalist would be a person who loves [[2]](https://youtu.be/NvLlpY9vd9E#t=352) nature. Just as a materialist ought to be a person who loves material, but certainly [[3]](https://youtu.be/NvLlpY9vd9E#t=360) isn't. With what is called a philosophy of scientific naturalism, "naturalism" is used [[4]](https://youtu.be/NvLlpY9vd9E#t=364) in a negative way. It has nothing to do with being natural; it has something to do with [[5]](https://youtu.be/NvLlpY9vd9E#t=374) being not supernatural, merely natural. All sorts of phrases were coined in that epoch [[6]](https://youtu.be/NvLlpY9vd9E#t=380) which I would call "put-down phrases." Freud spoke of the basic psychic energy as "libido," [[7]](https://youtu.be/NvLlpY9vd9E#t=387) which means "blind lust." People like Ernst Haeckel spoke of the universe as being a manifestation [[8]](https://youtu.be/NvLlpY9vd9E#t=396) of blind energy. Think of that put-down word "blind." We also speak of unconscious [[9]](https://youtu.be/NvLlpY9vd9E#t=404) mental mechanisms, and the very word "unconscious" as being the deeper aspect of our psyche is [[10]](https://youtu.be/NvLlpY9vd9E#t=413) a negative word and a put-down word. To say that you are functioning as a rational [[11]](https://youtu.be/NvLlpY9vd9E#t=421) ego with values and with a capacity to love is simply the epiphenomenon of a purely mechanical [[12]](https://youtu.be/NvLlpY9vd9E#t=429) process. Too bad. As a result of this so-called naturalism, we began to put up the most whopping [[13]](https://youtu.be/NvLlpY9vd9E#t=441) fight against nature that was ever engaged in. That fight is an expression of our [[14]](https://youtu.be/NvLlpY9vd9E#t=449) fury and of a feeling of being left out. The technological experiment which became [[15]](https://youtu.be/NvLlpY9vd9E#t=455) possible as a result of the mechanical sciences has largely been conducted in a spirit of [[16]](https://youtu.be/NvLlpY9vd9E#t=464) rage. The results are evident all around us. [[17]](https://youtu.be/NvLlpY9vd9E#t=471)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=346'>05:46</a>: Interestingly enough, when in the nineteenth century we switched our common sense from<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=352'>05:52</a>: supernaturalism to naturalism, one would think that a naturalist would be a person who love<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=360'>06:00</a>: nature. Just as a materialist ought to be a person who loves material but certainly<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=364'>06:04</a>: isn't. With what is called a philosophy of scientific naturalism. Naturalism is used<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=374'>06:14</a>: in a negative way. It has nothing to do with being natural it has something to do with<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=380'>06:20</a>: being not supernatural, Merely natural. And all sorts of phrases with coined in that epoch<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=387'>06:27</a>: which I would call put down phrases. Freud spoke of the basic psychic energy as Libido,<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=396'>06:36</a>: which means blind lust. People like Ernst Hegel spoke of the universe as being a manifestation<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=404'>06:44</a>: of blind energy. Think of that put down word blind. And therefore we also speak of unconscious<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=413'>06:53</a>: mental mechanisms. And the very word unconscious as being the deeper aspect of our psyche is<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=421'>07:01</a>: a negative word and a put down word. So is to say what you are functioning as a rational<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=429'>07:09</a>: ego with values and with a capacity to love, is simply the epiphenomenon of a purely mechanical<br>13. <a href='https://youtu.be/NvLlpY9vd9E#t=441'>07:21</a>: process. To bad. So as a result of this so-called naturalism we began to put up the most whopping<br>14. <a href='https://youtu.be/NvLlpY9vd9E#t=449'>07:29</a>: fight against nature that was ever engaged in. And that fight is an expression of our<br>15. <a href='https://youtu.be/NvLlpY9vd9E#t=455'>07:35</a>: fury and of a feeling of being left out. So that the technological experiment which became<br>16. <a href='https://youtu.be/NvLlpY9vd9E#t=464'>07:44</a>: possible as a result of the mechanical Sciences has largely been conducted in a spirit of<br>17. <a href='https://youtu.be/NvLlpY9vd9E#t=471'>07:51</a>: rage. And the results are evident all around us. Here in Palm Springs. You are gradually</details>

Here in Palm Springs, you are gradually getting all the smog from Los Angeles, this great cloud of poisonous gas put up by a city [[1]](https://youtu.be/NvLlpY9vd9E#t=481) which is exemplary in this whole civilized world for fouling its own nest. Perhaps only [[2]](https://youtu.be/NvLlpY9vd9E#t=489) Calcutta could be a bit worse, or some such terrible slum. But we have done it by technology, [[3]](https://youtu.be/NvLlpY9vd9E#t=499) by ruthless beating about of nature without consideration for what the scientist would [[4]](https://youtu.be/NvLlpY9vd9E#t=510) call our ecology. [[5]](https://youtu.be/NvLlpY9vd9E#t=520)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=481'>08:01</a>: getting all the smog from Los Angeles. This great cloud of poisonous gas put up by a city<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=489'>08:09</a>: which is exemplary in this whole civilized world for fouling it's own nest. Perhaps only<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=499'>08:19</a>: Calcutta could be a bit worse. Or some such terrible slum. But we have done it by technology.<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=510'>08:30</a>: By ruthless beating about of nature without consideration for what the scientist would<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=520'>08:40</a>: call our ecology.</details>

The only way that I can see of our effectively changing direction is through a transformation of the feeling [[1]](https://youtu.be/NvLlpY9vd9E#t=593)that we have of our own existence and of what we mean by "I". The reason for this is simply [[2]](https://youtu.be/NvLlpY9vd9E#t=603)that all kinds of intelligent and even powerful people like, say, Laurance Rockefeller who [[3]](https://youtu.be/NvLlpY9vd9E#t=612)are interested in ecology and in conservation of our natural resources, can scream [[4]](https://youtu.be/NvLlpY9vd9E#t=618)their heads off but nobody pays any attention. There is as yet no really serious program [[5]](https://youtu.be/NvLlpY9vd9E#t=624)at the government level to do anything radical about the pollution of water, the waste of [[6]](https://youtu.be/NvLlpY9vd9E#t=632)water, pollution of air and the general ravaging of the United States of America. I'm amazed [[7]](https://youtu.be/NvLlpY9vd9E#t=639)that congressmen can pass a bill imposing severe penalties on anyone who burns the American [[8]](https://youtu.be/NvLlpY9vd9E#t=648)flag, whereas they are responsible for burning that for which the flag stands: The United [[9]](https://youtu.be/NvLlpY9vd9E#t=654)States as a territory, as a people, and as a biological manifestation. That is an example [[10]](https://youtu.be/NvLlpY9vd9E#t=660)of our perennial confusion of symbols with realities, which is in a way at the heart [[11]](https://youtu.be/NvLlpY9vd9E#t=670)of the trouble because what we think of as "I" is much more a symbol than it is a reality. [[12]](https://youtu.be/NvLlpY9vd9E#t=678)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=593'>09:53</a>: can see of our effectively changing direction is through a transformation of the feeling<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=603'>10:03</a>: that we have of our own existence and of what we mean by I. The reason for this is simply<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=612'>10:12</a>: that all kinds of intelligent and even powerful people like, say, Laurance Rockefeller who<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=618'>10:18</a>: are interested in ecology and in conservation of our natural resources, they can scream<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=624'>10:24</a>: their heads off but nobody pays any attention. There is as yet no really serious program<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=632'>10:32</a>: at the government level to do anything radical about the pollution of water, the waste of<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=639'>10:39</a>: water, pollution of air and the general ravaging of the United States of America. I'm amazed<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=648'>10:48</a>: that congressman can pass a bill imposing severe penalties on anyone who burns the American<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=654'>10:54</a>: flag, whereas they are responsible for burning that for which the flag stands. The United<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=660'>11:00</a>: States as a territory, as a people, and as a biological manifestation. That is an example<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=670'>11:10</a>: of our perennial confusion of symbols with realities. Which is in a way at the heart<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=678'>11:18</a>: of the trouble because what we think of as I is much more a symbol than it is a reality.</details>

The living organism, the whole mind-body, is much more than anything we mean by "I". "I" [[1]](https://youtu.be/NvLlpY9vd9E#t=687)largely stands for your personality, your role in life. And the very word "person", as [[2]](https://youtu.be/NvLlpY9vd9E#t=696)you probably know, comes from the Latin "persona" - a word originally used for the mask worn by [[3]](https://youtu.be/NvLlpY9vd9E#t=705)actors in Graeco-Roman drama. That through which sound comes, because the mask had a [[4]](https://youtu.be/NvLlpY9vd9E#t=713)megaphonic mouth to carry the voice in open-air theaters. So when you speak of being a [[5]](https://youtu.be/NvLlpY9vd9E#t=719)real person, it really means being a genuine fake. Because the personality is only the [[6]](https://youtu.be/NvLlpY9vd9E#t=726)front. What is behind it? Well, of course, the organism is behind it. The whole organism. [[7]](https://youtu.be/NvLlpY9vd9E#t=734)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=687'>11:27</a>: The living organism, the whole mind-body, is much more than anything we mean by I. I<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=696'>11:36</a>: largely stands for your personality. Your role in life. And the very word person, as<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=705'>11:45</a>: you probably know, comes from the Latin persona. A word originally used for the mask worn by<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=713'>11:53</a>: actors in Graeco-Roman drama. That through which sound comes, because the mask had a<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=719'>11:59</a>: mega phonic mouth to carry the voice an open air theaters. So when you speak of being a<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=726'>12:06</a>: real person, it really means being a genuine fake. Because the personality is only the<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=734'>12:14</a>: front. What is behind it? Well of course the organism is behind it. The whole organism.</details>

We must be very careful not to confuse the organism with various symbols that we [[1]](https://youtu.be/NvLlpY9vd9E#t=744)have for it, because those symbols can be extremely misleading. If we say the organism [[2]](https://youtu.be/NvLlpY9vd9E#t=751)is the body, what we usually mean by the body is an impoverished meaning. When we speak [[3]](https://youtu.be/NvLlpY9vd9E#t=757)of "my body" - that is to say my vehicle, my physical automobile - that is an unenriched meaning [[4]](https://youtu.be/NvLlpY9vd9E#t=767)of the word body. Because what you really are as a body, as a living organism, is not [[5]](https://youtu.be/NvLlpY9vd9E#t=777)some sort of separate existence coated by a skin which divides you from the rest of [[6]](https://youtu.be/NvLlpY9vd9E#t=787)the world. Shakespeare has King John saying to Hubert: "Within this wall of flesh there [[7]](https://youtu.be/NvLlpY9vd9E#t=795)is a soul counts thee her creditor." Within this wall of flesh: the skin considered as [[8]](https://youtu.be/NvLlpY9vd9E#t=802)a barrier. When actually, from a biological point of view, the human skin and all skins [[9]](https://youtu.be/NvLlpY9vd9E#t=809)are osmotic membranes. You know when you get something by osmosis, by sort of soaking it [[10]](https://youtu.be/NvLlpY9vd9E#t=816)in. So in the same way, one's skin is a spongy construction full of holes, full of communicators, [[11]](https://youtu.be/NvLlpY9vd9E#t=824)nerve endings. And your skin is simply a vibrating membrane through which the so-called external [[12]](https://youtu.be/NvLlpY9vd9E#t=832)world flows into you and through you. [[13]](https://youtu.be/NvLlpY9vd9E#t=842)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=744'>12:24</a>: And we must be very careful not to confuse the organism with various symbols that we<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=751'>12:31</a>: have for it, because those symbols can be extremely misleading. If we say the organism<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=757'>12:37</a>: is the body, what we usually mean by the body, is an impoverished meaning. When we speak<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=767'>12:47</a>: of my body that is to say my vehicle, my physical automobile. That is an unenriched meaning<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=777'>12:57</a>: of the word body. Because what you really are as a body, as a living organism, is not<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=787'>13:07</a>: some sort of separate existence coated by a skin which divides you from the rest of<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=795'>13:15</a>: the world. Shakespeare has King John saying to Hubert: "Within this wall of flesh there<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=802'>13:22</a>: is a soul counts thee her creditor." Within this wall of flesh: the skin considered as<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=809'>13:29</a>: a barrier. When actually, from a biological point of view, the human skin and all skins<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=816'>13:36</a>: are osmotic membranes. You know when you get something by osmosis. By sort of soaking it<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=824'>13:44</a>: in. So in the same way one's skin is a spongy construction full of holes. Full of communicators,<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=832'>13:52</a>: nerve endings. And your skin is simply a vibrating membrane through which the so-called external<br>13. <a href='https://youtu.be/NvLlpY9vd9E#t=842'>14:02</a>: world flows into you and through you. So that you yourself actually are not so much an entity</details>

So you yourself actually are not so much an entity that moves around in an environment. You are much more like a whirlpool in a stream. And, [[1]](https://youtu.be/NvLlpY9vd9E#t=855)as you know, the whirlpool is constant only in its doing, that is to say in its whirling. [[2]](https://youtu.be/NvLlpY9vd9E#t=871)And you could recognize individual whirlpools in a stream. But the water is flowing through [[3]](https://youtu.be/NvLlpY9vd9E#t=877)them all the time. They are never the same but for a second. And so it is also with us. [[4]](https://youtu.be/NvLlpY9vd9E#t=884)Or imagine it in another way: supposing you have a rope and one foot of the rope is made [[5]](https://youtu.be/NvLlpY9vd9E#t=892)of hemp, one foot of it is made of cotton, one foot of it is made of silk, one foot of [[6]](https://youtu.be/NvLlpY9vd9E#t=897)it is made of nylon and so on. Now tie a simple knot in the rope. Now move the knot along [[7]](https://youtu.be/NvLlpY9vd9E#t=902)the rope. And one minute it will be hemp, the next cotton, next silk, next nylon and [[8]](https://youtu.be/NvLlpY9vd9E#t=910)so on. Same knot. It will be recognizable as a continuing knot, as that knot. The knot [[9]](https://youtu.be/NvLlpY9vd9E#t=918)in that rope. But the constitution of it will change as it moves. And so our constitution [[10]](https://youtu.be/NvLlpY9vd9E#t=924)is changing constantly. [[11]](https://youtu.be/NvLlpY9vd9E#t=931)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=855'>14:15</a>: that moves around in an environment. You are much more like a whirlpool in a stream. And,<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=871'>14:31</a>: as you know, the whirlpool is constant only in its doing, that is to say in its whirling.<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=877'>14:37</a>: And you could recognize individual whirligigs in a stream. But the water is flowing through<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=884'>14:44</a>: them all the time. They are never the same but for a second. And so it is also with us.<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=892'>14:52</a>: Or imagine it in another way supposing you have a rope and one foot of the rope is made<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=897'>14:57</a>: of hemp, one foot of it is made of cotton, one foot of it is made of silk, one foot of<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=902'>15:02</a>: it is made of nylon and so on. Now tie a simple knot in the rope. Now move the knot along<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=910'>15:10</a>: the rope. And one minute it will be hemp, the next cotton, next silk, next nylon and<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=918'>15:18</a>: so on. Same knot. It will be recognizable as a continuing knot, as that knot. The knot<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=924'>15:24</a>: in that rope. But the constitution of it will change as it moves. And so our constitution<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=931'>15:31</a>: is changing constantly. Imagine for example a university. The student body undergraduate</details>

Imagine for example a university. The student body undergraduate [[1]](https://youtu.be/NvLlpY9vd9E#t=937)changes every four years. The faculty changes every so often. The buildings keep changing [[2]](https://youtu.be/NvLlpY9vd9E#t=945)more and more. What constitutes the University of California? It certainly isn't the faculty, [[3]](https://youtu.be/NvLlpY9vd9E#t=952)it isn't the students, it isn't the governors, it isn't the administration, it isn't the [[4]](https://youtu.be/NvLlpY9vd9E#t=956)buildings. What is it? Why, a doing. A behavior. A university-ing process of study and experiment [[5]](https://youtu.be/NvLlpY9vd9E#t=965)and so on. So it is exactly the same with you. You flow. You are a process. [[6]](https://youtu.be/NvLlpY9vd9E#t=979)But how do we draw the line about this process and its relationship to all other processes? [[7]](https://youtu.be/NvLlpY9vd9E#t=984)We find that a very difficult thing to do the more you think about it. [[8]](https://youtu.be/NvLlpY9vd9E#t=991)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=937'>15:37</a>: changes every four years. The faculty changes every so often. The buildings keep changing<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=945'>15:45</a>: more and more. What constitutes the University of California? It certainly isn't the faculty,<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=952'>15:52</a>: it isn't the students, it isn't the governors, it isn't the administration, it isn't the<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=956'>15:56</a>: buildings. What is it? Why a doing. A behavior. A university-ing process of study and experiment<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=965'>16:05</a>: and so on. So it is exactly the same with you. You flow. You are a process.<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=979'>16:19</a>: But how do we draw the line about this process and its relationship to all other processes?<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=984'>16:24</a>: We find that a very difficult thing to do the more you think about it. If you really<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=991'>16:31</a>: felt with your whole organism, instead of just with that part of it called conscious</details>

If you really felt with your whole organism, instead of just with that part of it called conscious [[1]](https://youtu.be/NvLlpY9vd9E#t=996)attention, you would become aware of this flowing fact. And you would get a very strange [[2]](https://youtu.be/NvLlpY9vd9E#t=1005)feeling which at first might frighten you. It is possible of course to have this feeling. [[3]](https://youtu.be/NvLlpY9vd9E#t=1011)And the feeling is like this: You would not be quite sure how to interpret it. You might [[4]](https://youtu.be/NvLlpY9vd9E#t=1021)feel that you yourself were doing everything else that's happening. That would be one way [[5]](https://youtu.be/NvLlpY9vd9E#t=1027)of feeling. The other way of feeling it would be that you are doing nothing at all, but [[6]](https://youtu.be/NvLlpY9vd9E#t=1034)that everything else is doing you. And you would feel completely passive like a puppet [[7]](https://youtu.be/NvLlpY9vd9E#t=1040)on the end of strings. Although on the other hand if you got the feeling that you were [[8]](https://youtu.be/NvLlpY9vd9E#t=1046)doing it all you would feel like God Almighty. [[9]](https://youtu.be/NvLlpY9vd9E#t=1054)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=996'>16:36</a>: attention, you would become aware of this flowing fact. And you would get a very strange<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=1005'>16:45</a>: feeling which at first might frighten you. It is possible of course to have this feeling.<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=1011'>16:51</a>: And the feeling is like this: You would not be quite sure how to interpret it. You might<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=1021'>17:01</a>: feel that you yourself were doing everything else that's happening. That would be one way<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=1027'>17:07</a>: of feeling. The other way of feeling it would be that you are doing nothing at all. But<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=1034'>17:14</a>: that everything else is doing you. And you would feel completely passive like a puppet<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=1040'>17:20</a>: on the end of strings. Although on the other hand if you got the feeling that you were<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=1046'>17:26</a>: doing it all you would feel like God Almighty. It is very easy for our consciousness to slip<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=1054'>17:34</a>: into this state of sensation. It can happen spontaneously, like measles. It can happen</details>

It is very easy for our consciousness to slip into this state of sensation. It can happen spontaneously, like measles. It can happen [[1]](https://youtu.be/NvLlpY9vd9E#t=1064)by training, as when someone practices yoga. It can happen chemically as when certain drugs [[2]](https://youtu.be/NvLlpY9vd9E#t=1071)are taken. And one has to be very careful about this feeling because it's enormously [[3]](https://youtu.be/NvLlpY9vd9E#t=1081)easy to misinterpret. Either as being omnipotent, being God in the personal, literal sense, [[4]](https://youtu.be/NvLlpY9vd9E#t=1089)or as being helpless and merely driven. Now what should be understood is that both these [[5]](https://youtu.be/NvLlpY9vd9E#t=1098)ways of feeling are right. Only they must both be taken together. To be simultaneously [[6]](https://youtu.be/NvLlpY9vd9E#t=1106)omnipotent and helpless. These are two poles, opposite poles of one and the same state. [[7]](https://youtu.be/NvLlpY9vd9E#t=1116)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=1064'>17:44</a>: by training. As when someone practices yoga. It can happen chemically as when certain drugs<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=1071'>17:51</a>: are taken. And one has to be very careful about this feeling because it's enormously<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=1081'>18:01</a>: easy to misinterpret. Either as being omnipotent, being God in the personal, literal sense.<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=1089'>18:09</a>: Or as being helpless and merely driven. Now what should be understood is that both these<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=1098'>18:18</a>: ways of feeling are right. Only they must both be taken together. To be simultaneously<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=1106'>18:26</a>: omnipotent and helpless. These are two poles, opposite poles of one and the same state.<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=1116'>18:36</a>: Because the message that is coming through, and that we find difficulty in understanding,</details>

Because the message that is coming through, and that we find difficulty in understanding, [[1]](https://youtu.be/NvLlpY9vd9E#t=1121)because it's contrary to our common sense, contrary to our whole history and conditioning, [[2]](https://youtu.be/NvLlpY9vd9E#t=1126)the message that's coming through is: you as a living organism and all that is going [[3]](https://youtu.be/NvLlpY9vd9E#t=1132)on in your environment constitute a single process. What in physics we would call a unified [[4]](https://youtu.be/NvLlpY9vd9E#t=1138)field. A single process like a pattern but you know any pattern has all sorts of subsidiary [[5]](https://youtu.be/NvLlpY9vd9E#t=1145)wiggles in it. Like the organism itself is a unified pattern but it's full of wiggles. [[6]](https://youtu.be/NvLlpY9vd9E#t=1152)All sorts of tubes and organs and bones and nerves and so on. Working in this way. [[7]](https://youtu.be/NvLlpY9vd9E#t=1161)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=1121'>18:41</a>: because it's contrary to our common sense, contrary to our whole history and conditioning,<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=1126'>18:46</a>: the message that's coming through is: you as a living organism and all that is going<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=1132'>18:52</a>: on in your environment constitute a single process. What in physics we would call a unified<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=1138'>18:58</a>: field. A single process like a pattern but you know any pattern has all sorts of subsidiary<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=1145'>19:05</a>: wiggles in it. Like the organism itself is a unified pattern but it's full of wiggles.<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=1152'>19:12</a>: All sorts of tubes and organs and bones and nerves and so on. Working in this way. You<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=1161'>19:21</a>: know the body doesn't have a boss. We could pitch a big argument. Who is really the top</details>

But now let's take the argument to the side of the brain. The brain says: "Oh [[1]](https://youtu.be/NvLlpY9vd9E#t=1211)no no. I'm not. Just because I arrived late doesn't mean that I'm unimportant. I was being [[2]](https://youtu.be/NvLlpY9vd9E#t=1219)gotten ready for. Because I am the thing that is the flower at the top of this thing. And [[3]](https://youtu.be/NvLlpY9vd9E#t=1224)this tube with stomachs in it and things below was preparing for me and the stomach is my [[4]](https://youtu.be/NvLlpY9vd9E#t=1233)servant. It is doing all the dirty work and getting energy to put currents through my [[5]](https://youtu.be/NvLlpY9vd9E#t=1239)wonderful circuits. So that by the creation of all the goods of the mind of the Arts and [[6]](https://youtu.be/NvLlpY9vd9E#t=1246)Sciences and religion and philosophy and so on I shall be the true head of man." Well, both [[7]](https://youtu.be/NvLlpY9vd9E#t=1253)arguments are right. Because you have a relationship between stomach and brain, which is a sort [[8]](https://youtu.be/NvLlpY9vd9E#t=1263)of polarity. The one exists for the other. It's like when you prop up two sticks against [[9]](https://youtu.be/NvLlpY9vd9E#t=1272)each other they will stand up so long as they lean on each other. Take one away and the [[10]](https://youtu.be/NvLlpY9vd9E#t=1280)other collapses. So chop off the head and the stomach is finished. Take out the stomach [[11]](https://youtu.be/NvLlpY9vd9E#t=1284)and the head is finished. So this is the way all organic life proceeds.
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=1211'>20:11</a>: brain. But now let's take the argument to the side of the brain. The brain says: "Oh<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=1219'>20:19</a>: no no. I'm not. Just because I arrived late doesn't mean that I'm unimportant. I was being<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=1224'>20:24</a>: gotten ready for. Because I am the thing that is the flower at the top of this thing. And<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=1233'>20:33</a>: this tube with stomachs in it and things below was preparing for me and the stomach is my<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=1239'>20:39</a>: servant. It is doing all the dirty work and getting energy to put currents through my<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=1246'>20:46</a>: wonderful circuits. So that by the creation of all the goods of the mind of the Arts and<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=1253'>20:53</a>: Sciences and religion and philosophy and so on I shall be the true head of man. Well both<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=1263'>21:03</a>: arguments are right. Because you have a relationship between stomach and brain, which is a sort<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=1272'>21:12</a>: of polarity. The one exists for the other. It's like when you prop up two sticks against<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=1280'>21:20</a>: each other they will stand up so long as they lean on each other. Take one away and the<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=1284'>21:24</a>: other collapses. So chop off the head and the stomach is finished. Take out the stomach</details>

It's different [[1]](https://youtu.be/NvLlpY9vd9E#t=1288)with mechanical life. Because the mechanism must invariably have a boss. The man who puts [[2]](https://youtu.be/NvLlpY9vd9E#t=1297)the machine together, the person who constructs the computer, who designs it, who asks it [[3]](https://youtu.be/NvLlpY9vd9E#t=1307)questions, who programs it. He is the boss. But organisms don't have bosses. They are [[4]](https://youtu.be/NvLlpY9vd9E#t=1313)essentially, I would say, democratic arrangements. Where, somehow, in a marvelous way, an enormous [[5]](https://youtu.be/NvLlpY9vd9E#t=1321)company of cells are working together. But that isn't the way, the body wasn't sort of: [[6]](https://youtu.be/NvLlpY9vd9E#t=1333)one day a lot of cells all crept together and said: We are a body. That does sometimes [[7]](https://youtu.be/NvLlpY9vd9E#t=1340)happen in the biological domain, but much rather this: when you watch the gestation [[8]](https://youtu.be/NvLlpY9vd9E#t=1344)of a mammal, you see first of all a very simple little organism. Which swells. And as it swells, [[9]](https://youtu.be/NvLlpY9vd9E#t=1349)it becomes more and more complicated from within. No parts are added, nothing is screwed [[10]](https://youtu.be/NvLlpY9vd9E#t=1358)on, there's no welding done or anything like that. It bulges and of course it does absorb [[11]](https://youtu.be/NvLlpY9vd9E#t=1362)material, but it transforms it. But all of it works together at once like the legs of [[12]](https://youtu.be/NvLlpY9vd9E#t=1369)a centipede. Like you work all together at once.
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=1288'>21:28</a>: and the head is finished. So this is the way all organic life proceeds. It's different<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=1297'>21:37</a>: with mechanical life. Because the mechanism must invariably have a boss. The man who puts<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=1307'>21:47</a>: the machine together, the person who constructs the computer, who designs it, who asks it<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=1313'>21:53</a>: questions, who programs it. He is the boss. But organisms don't have bosses. They are<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=1321'>22:01</a>: essentially, I would say, democratic arrangements. Where, somehow, in a marvelous way, an enormous<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=1333'>22:13</a>: company of cells are working together. But that isn't the way, the body wasn't sort of:<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=1340'>22:20</a>: one day a lot of cells all crept to together and said: We are a body. That does sometimes<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=1344'>22:24</a>: happen in the biological domain, but much rather this: when you watch the gestation<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=1349'>22:29</a>: of a mammal, you see first of all a very simple little organism. Which swells. And as it swells,<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=1358'>22:38</a>: it becomes more and more complicated from within. No parts are added, nothing is screwed<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=1362'>22:42</a>: on, there's no welding done or anything like that. It bulges and of course it does absorb<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=1369'>22:49</a>: material, but it transforms it. But all of it works together at once like the legs of</details>

For, you see, when we come down to it, [[1]](https://youtu.be/NvLlpY9vd9E#t=1373)you think you decide things, but you don't know how you do it. [[2]](https://youtu.be/NvLlpY9vd9E#t=1383)How do you open and close your hand? And you can decide "I will now open my hand" and do it, but [[3]](https://youtu.be/NvLlpY9vd9E#t=1388)you don't know how it's done. And yet in a sense you do know how it's done because you [[4]](https://youtu.be/NvLlpY9vd9E#t=1393)say "I know how to open my hand". But you don't know in words. You can't explain it. Still [[5]](https://youtu.be/NvLlpY9vd9E#t=1399)less can you explain how you see, still less can you explain how you are conscious. How [[6]](https://youtu.be/NvLlpY9vd9E#t=1408)are you an ego? Well you don't know because the springs of being conscious, of being an [[7]](https://youtu.be/NvLlpY9vd9E#t=1414)ego are outside the surveillance of consciousness. They're somehow underneath. And that lets [[8]](https://youtu.be/NvLlpY9vd9E#t=1422)the cat out of the bag at once, because you see that what is "I" is something very very [[9]](https://youtu.be/NvLlpY9vd9E#t=1430)much deeper than the superficial consciousness. And what you call "I" in the sense of the voluntary [[10]](https://youtu.be/NvLlpY9vd9E#t=1436)willing center, ego, has very little to do with it. You are just a watchman on top of [[11]](https://youtu.be/NvLlpY9vd9E#t=1445)the mast or a radar on a ship that is scanning the environment by conscious attention looking [[12]](https://youtu.be/NvLlpY9vd9E#t=1451)out for trouble. Or looking for food. The real you is much too complicated to think [[13]](https://youtu.be/NvLlpY9vd9E#t=1457)about.
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=1373'>22:53</a>: a centipede. Like you work all together at once. For, you see, when we come down to it,<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=1383'>23:03</a>: you think you decide things, but you don't know how you do it.<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=1388'>23:08</a>: How do you open and close your hand? And you can decide I will now open my and do it, but<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=1393'>23:13</a>: you don't know how it's done. And yet in a sense you do know how it's done because you<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=1399'>23:19</a>: say I know how to open my hand. But you don't know in words. You can't explain it. Still<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=1408'>23:28</a>: less can you explain how you see, still less can you explain how you are conscious. How<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=1414'>23:34</a>: are you an ego? Well you don't know because the springs of being conscious, of being an<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=1422'>23:42</a>: ego are outside the surveillance of consciousness. They're somehow underneath. And that lets<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=1430'>23:50</a>: the cat out of the bag at once, because you see that what is I is something very very<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=1436'>23:56</a>: much deeper than the superficial consciousness. And what you call I in the sense of the voluntary<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=1445'>24:05</a>: willing center, ego, has very little to do with it. You are just a watchman on top of<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=1451'>24:11</a>: the mast or a radar on a ship that is scanning the environment by conscious attention looking<br>13. <a href='https://youtu.be/NvLlpY9vd9E#t=1457'>24:17</a>: out for trouble. Or looking for food. The real you is much too complicated to think</details>

Supposing when you woke up in the morning, you had to switch yourself on. That is to [[1]](https://youtu.be/NvLlpY9vd9E#t=1467)say, you had to, by an act of conscious attention, to go through your brain and turn on all the [[2]](https://youtu.be/NvLlpY9vd9E#t=1476)synapses necessary for wakeful life. It'd take you hours. Supposing you really had to [[3]](https://youtu.be/NvLlpY9vd9E#t=1481)be conscious of all the details involved in walking or in breathing or in circulation [[4]](https://youtu.be/NvLlpY9vd9E#t=1490)of the blood. You'd never get around to it. So you see when we inspect the physical world [[5]](https://youtu.be/NvLlpY9vd9E#t=1495)with conscious attention the first thing that strikes us is that the physical world is extraordinarily [[6]](https://youtu.be/NvLlpY9vd9E#t=1505)complicated. How can it possibly be organized. But actually the physical world is not complicated [[7]](https://youtu.be/NvLlpY9vd9E#t=1509)at all. What is complicated is the task of trying to describe it in words. Or of trying [[8]](https://youtu.be/NvLlpY9vd9E#t=1520)to figure it out in numbers. Because that is analogous to the task of, say, removing [[9]](https://youtu.be/NvLlpY9vd9E#t=1530)the water from the Pacific Ocean into the Atlantic with a beer mug. We can only take [[10]](https://youtu.be/NvLlpY9vd9E#t=1537)one mug at a time. And so we say in popular speech you can only think of one thing at [[11]](https://youtu.be/NvLlpY9vd9E#t=1544)a time. That's not exactly true but what it reveals is that thinking, that conscious thought [[12]](https://youtu.be/NvLlpY9vd9E#t=1552)is a kind of calculus in which we understand things bit by bit. And it leads us into the [[13]](https://youtu.be/NvLlpY9vd9E#t=1561)superstition that things really are bits.
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=1467'>24:27</a>: about. Supposing when you woke up in the morning, you had to switch yourself on. That is to<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=1476'>24:36</a>: say, you had to, by an act of conscious attention, to go through your brain and turn on all the<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=1481'>24:41</a>: synapses necessary for wakeful life. It'd take you hours. Supposing you really had to<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=1490'>24:50</a>: be conscious of all the details involved in walking or in breathing or in circulation<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=1495'>24:55</a>: of the blood. You'd never get around to it. So you see when we inspect the physical world<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=1505'>25:05</a>: with conscious attention the first thing that strikes us is that the physical world is extraordinarily<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=1509'>25:09</a>: complicated. How can it possibly be organized. But actually the physical world is not complicated<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=1520'>25:20</a>: at all. What is complicated is the task of trying to describe it in words. Or of trying<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=1530'>25:30</a>: to figure it out in numbers. Because that is analogous to the task of, say, removing<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=1537'>25:37</a>: the water from the Pacific Ocean into the Atlantic with a beer mug. We can only take<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=1544'>25:44</a>: one mug at a time. And so we say in popular speech you can only think of one thing at<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=1552'>25:52</a>: a time. That's not exactly true but what it reveals is that thinking, that conscious thought<br>13. <a href='https://youtu.be/NvLlpY9vd9E#t=1561'>26:01</a>: is a kind of calculus in which we understand things bit by bit. And it leads us into the</details>

Now, when you eat chicken of course you have [[1]](https://youtu.be/NvLlpY9vd9E#t=1570)to bite it. And you take it in bits. And to make it easier to bite you order from the [[2]](https://youtu.be/NvLlpY9vd9E#t=1580)grocer a cut up fryer. But you don't get cut up fryers out of eggs. Because you see although [[3]](https://youtu.be/NvLlpY9vd9E#t=1587)we can speak of A chicken, AN egg, or A body, it is not actually a bit. It hasn't been bitten [[4]](https://youtu.be/NvLlpY9vd9E#t=1596)off except for purposes of thinking. Now. [[5]](https://youtu.be/NvLlpY9vd9E#t=1605)This is beautifully brought out in a passage from Whitehead, which I will read to you. [[6]](https://youtu.be/NvLlpY9vd9E#t=1615)He is discussing the nineteenth century philosophy of science, which I was just discussing, too. [[7]](https://youtu.be/NvLlpY9vd9E#t=1631)And He's saying in this philosophy: "All our impressions of nature are simply products [[8]](https://youtu.be/NvLlpY9vd9E#t=1658)of our minds. Nature gets credit which should in truth be reserved for ourselves. The rose [[9]](https://youtu.be/NvLlpY9vd9E#t=1682)for its scent, the nightingale for his song and the sun for his radiance. The poets are [[10]](https://youtu.be/NvLlpY9vd9E#t=1690)entirely mistaken. They should address their lyrics to themselves and should turn them [[11]](https://youtu.be/NvLlpY9vd9E#t=1696)into odes of self congratulation on the excellency of the human mind. Nature is a dull affair. [[12]](https://youtu.be/NvLlpY9vd9E#t=1701)Soundless, scentless, colorless. Merely the hurrying of material, endlessly, meaninglessly." [[13]](https://youtu.be/NvLlpY9vd9E#t=1707)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=1570'>26:10</a>: superstition that things really are bits. Now, when you eat chicken of course you have<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=1580'>26:20</a>: to bite it. And you take it in bits. And to make it easier to bite you order from the<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=1587'>26:27</a>: grocer a cut up fryer. But you don't get cut up fryers out of eggs. Because you see although<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=1596'>26:36</a>: we can speak of A chicken, AN egg, or A body, it is not actually a bit. It hasn't been bitten<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=1605'>26:45</a>: off except for purposes of thinking. Now.<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=1615'>26:55</a>: This is beautifully brought out in a passage from Whitehead, which I will read to you.<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=1631'>27:11</a>: He is discussing the nineteenth century philosophy of science, which I was just discussing, too.<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=1658'>27:38</a>: And He's saying in this philosophy: All our impressions of nature are simply products<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=1682'>28:02</a>: of our minds. Nature gets credit which should in truth be reserved for ourselves. The rose<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=1690'>28:10</a>: for its sent the nightingale for his song and the sun for his radiance. The poets are<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=1696'>28:16</a>: entirely mistaken. They should address their lyrics to themselves and should turn them<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=1701'>28:21</a>: into odes of self congratulation on the excellency of the human mind. Nature is a dull affair.<br>13. <a href='https://youtu.be/NvLlpY9vd9E#t=1707'>28:27</a>: Soundless, scentless, colorless. Merely the hurrying of material, endlessly, meaninglessly.</details>

"However you disguise it, this is the practical outcome of the characteristic scientific philosophy [[1]](https://youtu.be/NvLlpY9vd9E#t=1716)which closed the seventeenth century. In the first place we must note its astounding [[2]](https://youtu.be/NvLlpY9vd9E#t=1721)efficiency as a system of concepts for the organization of scientific research. In this [[3]](https://youtu.be/NvLlpY9vd9E#t=1727)respect, it is fully worthy of the genius of the century which produced it. It has held [[4]](https://youtu.be/NvLlpY9vd9E#t=1733)its own as the guiding principle of scientific studies ever since. It is still reigning. Every [[5]](https://youtu.be/NvLlpY9vd9E#t=1739)university in the world organizes itself in accordance with it. No alternative system [[6]](https://youtu.be/NvLlpY9vd9E#t=1745)of organizing the pursuit of scientific truth has been suggested. It is not only reigning, [[7]](https://youtu.be/NvLlpY9vd9E#t=1750)but it is without a rival. And yet it is quite unbelievable. This conception of the universe [[8]](https://youtu.be/NvLlpY9vd9E#t=1756)is surely framed in terms of high abstractions and the paradox only arises because we have [[9]](https://youtu.be/NvLlpY9vd9E#t=1765)mistaken our abstractions for concrete realities." He calls that, you see, the fallacy of misplaced [[10]](https://youtu.be/NvLlpY9vd9E#t=1772)concreteness. The attribution, in other words, to our bitty way of thinking to the world [[11]](https://youtu.be/NvLlpY9vd9E#t=1784)which we are biting. That is misplaced concreteness. The separations between things are abstract. [[12]](https://youtu.be/NvLlpY9vd9E#t=1795)They are concepts. In the same way, for example, as lines of latitude and longitude are concepts. [[13]](https://youtu.be/NvLlpY9vd9E#t=1804)Even though a Russian poet has recently made a beautiful poem about the world being like [[14]](https://youtu.be/NvLlpY9vd9E#t=1811)a ball carried in a net bag. But you never expect to trip over the equator when you cross [[15]](https://youtu.be/NvLlpY9vd9E#t=1817)it. And although it is something quite abstract and does not exist in nature, it's extremely [[16]](https://youtu.be/NvLlpY9vd9E#t=1827)useful for purposes of navigation. So in the same way, bit-ing and having words to describe [[17]](https://youtu.be/NvLlpY9vd9E#t=1834)particular events or particular wiggles in the universal pattern are very useful. But [[18]](https://youtu.be/NvLlpY9vd9E#t=1841)they are very dangerous when you confuse things with natural events. Because then you get [[19]](https://youtu.be/NvLlpY9vd9E#t=1848)into this sort of trouble.
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=1716'>28:36</a>: However you disguise it, this is the practical outcome of the characteristic scientific philosophy<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=1721'>28:41</a>: which closed even the seventeenth century. In the first place we must note its astounding<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=1727'>28:47</a>: efficiency as a system of concepts for the organization of scientific research. In this<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=1733'>28:53</a>: respect, it is fully worthy of the genius of the century which produced it. It has held<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=1739'>28:59</a>: its own as the guiding principle of scientific studies ever since. It is still raining. Every<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=1745'>29:05</a>: university in the world organizes itself in accordance with it. No alternative system<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=1750'>29:10</a>: of organizing the pursuit of scientific truth has been suggested. It is not only reigning,<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=1756'>29:16</a>: but it is without a rival. And yet it is quite unbelievable. This conception of the universe<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=1765'>29:25</a>: is surely framed in terms of high abstractions and the paradox only arises because we have<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=1772'>29:32</a>: mistaken our abstractions for concrete realities. He calls that, you see, the fallacy of misplaced<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=1784'>29:44</a>: concreteness. The attribution, in other words, to our bitty way of thinking to the world<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=1795'>29:55</a>: which we are biting. That is misplaced concreteness. The separations between things are abstract.<br>13. <a href='https://youtu.be/NvLlpY9vd9E#t=1804'>30:04</a>: They are concepts. In the same way, for example, as lines of latitude and longitude are concepts.<br>14. <a href='https://youtu.be/NvLlpY9vd9E#t=1811'>30:11</a>: Even though a Russian poet has recently made a beautiful poem about the world being like<br>15. <a href='https://youtu.be/NvLlpY9vd9E#t=1817'>30:17</a>: a ball carried in a net bag. But you never expect to trip over the equator when you cross<br>16. <a href='https://youtu.be/NvLlpY9vd9E#t=1827'>30:27</a>: it. And although it is something quite abstract and does not exist in nature, it's extremely<br>17. <a href='https://youtu.be/NvLlpY9vd9E#t=1834'>30:34</a>: useful for purposes of navigation. So in the same way, bit-ing and having words to describe<br>18. <a href='https://youtu.be/NvLlpY9vd9E#t=1841'>30:41</a>: particular events or particular wiggles in the universal pattern are very useful. But<br>19. <a href='https://youtu.be/NvLlpY9vd9E#t=1848'>30:48</a>: they are very dangerous when you confuse things with natural events. Because then you get</details>

Take the case of bees and flowers. Oh, we always use the bees and flowers to [[1]](https://youtu.be/NvLlpY9vd9E#t=1929)explain fundamental things about life but we're going to go deeper than sex this time. [[2]](https://youtu.be/NvLlpY9vd9E#t=1935)Fascinating thing about bees and flowers is they are very different looking things. A [[3]](https://youtu.be/NvLlpY9vd9E#t=1940)flower sits still and blooms and it smells, or stinks to be correct. The bee moves about [[4]](https://youtu.be/NvLlpY9vd9E#t=1946)and buzzes. But they are all one organism. You don't find flowers without bees, you don't [[5]](https://youtu.be/NvLlpY9vd9E#t=1954)find bees without flowers. They are just as much one as your head and your feet which [[6]](https://youtu.be/NvLlpY9vd9E#t=1963)also look very different. So in that sense you see we are one with the incredible complexity [[7]](https://youtu.be/NvLlpY9vd9E#t=1968)of processes and wiggles upon which we depend. Although to say "upon which we depend" is [[8]](https://youtu.be/NvLlpY9vd9E#t=1981)not quite accurate. Because that separates us from it as if I were hanging onto a beam [[9]](https://youtu.be/NvLlpY9vd9E#t=1989)and depending on it. It isn't like that. You don't depend on it, because it depends on [[10]](https://youtu.be/NvLlpY9vd9E#t=1995)you. It's a mutual arrangement. And it isn't that one bit of this sort of came first. Although [[11]](https://youtu.be/NvLlpY9vd9E#t=2001)that sometimes happens. But it's always there in potentiality, what came later.
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=1929'>32:09</a>: an engine. Take the case of bees and flowers. Oh, we always use the bees and flowers to<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=1935'>32:15</a>: explain fundamental things about life but we're going to go deeper than sex this time.<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=1940'>32:20</a>: Fascinating thing about bees and flowers is they are very different looking things. A<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=1946'>32:26</a>: flower sits still and blooms and it smells, or stinks to be correct. The bee moves about<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=1954'>32:34</a>: and buzzes. But they are all one organism. You don't find flowers without bees, you don't<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=1963'>32:43</a>: find bees without flowers. They are just as much one as your head and your feet which<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=1968'>32:48</a>: also look very different. So in that sense you see we are one with the incredible complexity<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=1981'>33:01</a>: of processes and wiggles upon which we depend. Although to say "upon which we depend" is<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=1989'>33:09</a>: not quite accurate. Because that separates us from it as if I were hanging onto a beam<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=1995'>33:15</a>: and depending on it. It isn't like that. You don't depend on it, because it depends on<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=2001'>33:21</a>: you. It's a mutual arrangement. And it isn't that one bit of this sort of came first. Although</details>

But it's [[1]](https://youtu.be/NvLlpY9vd9E#t=2013)rather in the same way that, when a flower opens, you see all the different petals extending [[2]](https://youtu.be/NvLlpY9vd9E#t=2021)simultaneously. Especially when you watch a fast motion movie of a flower opening. And [[3]](https://youtu.be/NvLlpY9vd9E#t=2028)so in the same way there is a simultaneous arrival, or evolution, of the human organism [[4]](https://youtu.be/NvLlpY9vd9E#t=2034)and the human environment and thus biologists speak about the evolution of an environment [[5]](https://youtu.be/NvLlpY9vd9E#t=2043)as well as the evolution of an organism in it. In other words, human beings could not [[6]](https://youtu.be/NvLlpY9vd9E#t=2050)have appeared on this planet until its temperature had lowered to a certain degree, until the [[7]](https://youtu.be/NvLlpY9vd9E#t=2055)atmosphere contained certain gases as a result of vegetative development. And then the environment [[8]](https://youtu.be/NvLlpY9vd9E#t=2060)became evolved enough for human beings to appear in it. Evolved enough? I'll say if [[9]](https://youtu.be/NvLlpY9vd9E#t=2069)something further: intelligent enough for the appearance in it of intelligent beings. [[10]](https://youtu.be/NvLlpY9vd9E#t=2077)For your environment is intelligent. Otherwise you couldn't be. You see, as Jesus said, you [[11]](https://youtu.be/NvLlpY9vd9E#t=2084)don't gather figs off thistles or grapes from thorns. You won't get pears off an apple tree. [[12]](https://youtu.be/NvLlpY9vd9E#t=2097)So you won't find people except on a people tree. And this planet, this solar system, [[13]](https://youtu.be/NvLlpY9vd9E#t=2107)this galaxy is people-ing in exactly the same way that an apple tree apples.
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2013'>33:33</a>: that sometimes happens. But it's always there in potentiality, what came later. But it's<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2021'>33:41</a>: rather in the same way that, when a flower opens, you see all the different petals extending<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2028'>33:48</a>: simultaneously. Especially when you watch a fast motion movie of a flower opening. And<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2034'>33:54</a>: so in the same way there is a simultaneous arrival, or evolution, of the human organism<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2043'>34:03</a>: and the human environment and thus biologists speak about the evolution of an environment<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=2050'>34:10</a>: as well as the evolution of an organism in it. In other words, human beings could not<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=2055'>34:15</a>: have appeared on this planet until its temperature had lowered to a certain degree, until the<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=2060'>34:20</a>: atmosphere contained certain gases as a result of vegetative development. And then the environment<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=2069'>34:29</a>: became evolved enough for human beings to appear in it. Evolved enough? I'll say if<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=2077'>34:37</a>: something further: intelligent enough for the appearance in it of intelligent beings.<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=2084'>34:44</a>: For your environment is intelligent. Otherwise you couldn't be. You see, as Jesus said, you<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=2097'>34:57</a>: don't gather figs off thistles or grapes from thorns. You won't get pears off an apple tree.<br>13. <a href='https://youtu.be/NvLlpY9vd9E#t=2107'>35:07</a>: So you won't find people except on a people tree. And this planet, this solar system,</details>

Put our existence into verb language, as distinct from noun language, and you're much closer [[1]](https://youtu.be/NvLlpY9vd9E#t=2122)to the point. You see nouns have the difficulty of designating things. Verbs designate process. [[2]](https://youtu.be/NvLlpY9vd9E#t=2127)Now everything is a process, really. When we speak of housing for houses, matting for [[3]](https://youtu.be/NvLlpY9vd9E#t=2137)maps, we're getting there. The Nootka Indians have a language in which there are no nouns. [[4]](https://youtu.be/NvLlpY9vd9E#t=2144)So they say: it houses. And then they add an adverb to show whether it houses religiously, [[5]](https://youtu.be/NvLlpY9vd9E#t=2152)homeyly or marketingly. And so they see the world as the flow. What is IT that houses. What [[6]](https://youtu.be/NvLlpY9vd9E#t=2162)is it that rains when we say it is raining? You see we always have a funny idea that to [[7]](https://youtu.be/NvLlpY9vd9E#t=2177)get a verb, that is to say, to get action you have to have an agent. Now this is the [[8]](https://youtu.be/NvLlpY9vd9E#t=2186)most ridiculous idea conceivable. How can a noun start a verb? How can a thing start [[9]](https://youtu.be/NvLlpY9vd9E#t=2193)an event? Because there's no action in a thing. Action can only come from action. Energy from [[10]](https://youtu.be/NvLlpY9vd9E#t=2201)energy. You can't get energy from a concept. Because nouns are all concepts, they're abstract, [[11]](https://youtu.be/NvLlpY9vd9E#t=2208)really. It's only verbs that are concrete. As the world is process.
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2122'>35:22</a>: Put our existence into verb language, as distinct from noun language, and you're much closer<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2127'>35:27</a>: to the point. You see nouns have the difficulty of designating things. Verbs designate process.<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2137'>35:37</a>: Now everything is a process, really. When we speak of housing for houses, matting for<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2144'>35:44</a>: maps, we're getting there. The Nootka Indians have a language in which there are no nouns.<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2152'>35:52</a>: So they say: it houses. And then they add an adverb to show whether it houses religiously,<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=2162'>36:02</a>: homeyly or marketingly. And so they see the world as the flow. What is IT that houses.What<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=2177'>36:17</a>: is it that rains when we say it is raining. You see we always have a funny idea that to<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=2186'>36:26</a>: get a verb, that is to say, to get action you have to have an agent. Now this is the<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=2193'>36:33</a>: most ridiculous idea conceivable. How can a noun start a verb? How can a thing start<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=2201'>36:41</a>: an event? Because there's no action in a thing. Action can only come from action. Energy from<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=2208'>36:48</a>: energy. You can't get energy from a concept. Because nouns are all concepts, they're abstract,</details>

Upon reflection, we find that what we thought of as "I" is merely a social construct, like the equator or an inch. [[1]](https://youtu.be/NvLlpY9vd9E#t=2371) [[2]](https://youtu.be/NvLlpY9vd9E#t=2378) [[3]](https://youtu.be/NvLlpY9vd9E#t=2392)To mistake it for something concrete is what Whitehead called the "fallacy of misplaced concreteness." [[4]](https://youtu.be/NvLlpY9vd9E#t=2409)Psychological self-exploration reveals this to be the case. [[5]](https://youtu.be/NvLlpY9vd9E#t=2419)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2371'>39:31</a>: Just look and see what has happened to us. If we go through everything that I've been<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2378'>39:38</a>: saying, we find, first of all, that the thing that we thought was I is nothing more than<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2392'>39:52</a>: a social institution. Just like the equator. Or an inch. And to mistake it, to reify it,<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2409'>40:09</a>: as Whitehead would say, is the fallacy of misplaced concreteness. It's a strictly a<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2419'>40:19</a>: hallucination. And any certain amount of psychological self-exploration shows this to be the case.</details>

What we truly are is the organism, which exists in a transactional interchange with the environment. [[1]](https://youtu.be/NvLlpY9vd9E#t=2434)It's not quite correct to say we're doing it and it's doing us, as it's all one unified process. [[2]](https://youtu.be/NvLlpY9vd9E#t=2448) [[3]](https://youtu.be/NvLlpY9vd9E#t=2455) [[4]](https://youtu.be/NvLlpY9vd9E#t=2462) [[5]](https://youtu.be/NvLlpY9vd9E#t=2468) [[6]](https://youtu.be/NvLlpY9vd9E#t=2476) [[7]](https://youtu.be/NvLlpY9vd9E#t=2487) [[8]](https://youtu.be/NvLlpY9vd9E#t=2494) [[9]](https://youtu.be/NvLlpY9vd9E#t=2502)From this process, what we thought was solid has disappeared. [[10]](https://youtu.be/NvLlpY9vd9E#t=2510)What we took for common sense and substance is just pattern. [[11]](https://youtu.be/NvLlpY9vd9E#t=2519)This can make one feel ghostly, as if easily blown away. [[12]](https://youtu.be/NvLlpY9vd9E#t=2529)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2434'>40:34</a>: But what we are is the organism. And what the organism is is a transactional interchange<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2448'>40:48</a>: between the organism and the environment. It's not quite correct to say you're doing<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2455'>40:55</a>: it and it's doing you because, you see, Whitehead, in describing the scientific philosophy, saying,<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2462'>41:02</a>: for example, that blue is entirely our projection on the sky, he's half right, you see. Those<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2468'>41:08</a>: scientists he's talking about are half right. But the part of it that is left out of consideration<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=2476'>41:16</a>: is this: True, you with your optical nerves and eyes transform the sky into the blue feeling.<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=2487'>41:27</a>: But without the sky you wouldn't have any optical nerves. It works both ways you see.<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=2494'>41:34</a>: Without the air whose density gives the blue effect. It's mutual. You do it and it does<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=2502'>41:42</a>: you. But that's, as I said, a two-way, a clumsy way of talking about: it's all one process,<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=2510'>41:50</a>: a unified process. And furthermore from this process there has disappeared what we thought<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=2519'>41:59</a>: was solidity. What we thought was common sense, substance and stuff- it's just pattern. And<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=2529'>42:09</a>: at once one feels sort of ghostly. As if you could be easily blown away. And that's why</details>

This is why Hindus call the universe Maya, meaning illusion. [[1]](https://youtu.be/NvLlpY9vd9E#t=2537)Maya is related to the Latin "ludere" (to play), and also means magic, creative power, art, and divine power. [[2]](https://youtu.be/NvLlpY9vd9E#t=2547) [[3]](https://youtu.be/NvLlpY9vd9E#t=2560) [[4]](https://youtu.be/NvLlpY9vd9E#t=2567) [[5]](https://youtu.be/NvLlpY9vd9E#t=2578)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2537'>42:17</a>: the Hindus call the universe the Maya. Which means the illusion. Don't forget illusion<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2547'>42:27</a>: is related to the Latin word ludere- to play. So the play. Big act. It also means magic.<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2560'>42:40</a>: As in a conjuror's creation of an illusion. It also means creative power. It also means<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2567'>42:47</a>: art. Finally it means the divine power. The Maya of the Lord. Lord is a bad translation<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2578'>42:58</a>: of Bhagavan. Just the Divine One. Lord means boss and the Hindus don't do it that way.</details>

Having established this perspective, the question becomes: How can we transform human consciousness so that people truly know and feel this reality, [[1]](https://youtu.be/NvLlpY9vd9E#t=2591) [[2]](https://youtu.be/NvLlpY9vd9E#t=2597) [[3]](https://youtu.be/NvLlpY9vd9E#t=2602) [[4]](https://youtu.be/NvLlpY9vd9E#t=2611)rather than just intellectually understanding it? [[5]](https://youtu.be/NvLlpY9vd9E#t=2617)Without such a transformation, we will not be fit to use technology responsibly. [[6]](https://youtu.be/NvLlpY9vd9E#t=2622)We will continue to use it in a hostile spirit towards the external world, wreaking havoc. [[7]](https://youtu.be/NvLlpY9vd9E#t=2625) [[8]](https://youtu.be/NvLlpY9vd9E#t=2632) [[9]](https://youtu.be/NvLlpY9vd9E#t=2638)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2591'>43:11</a>: But now you see, having arranged this general introduction, which I'm afraid will be familiar<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2597'>43:17</a>: to some of you who've attended my seminars before; but I've arranged this general introduction<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2602'>43:22</a>: to raise the question: All right. If that's the way it is. How on earth are we going to<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2611'>43:31</a>: arrange a transformation of man's consciousness so that he'll know it. Not just in theory<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2617'>43:37</a>: but something he feels in the same way as you feel what you take to be I at the present<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=2622'>43:42</a>: moment, confronting an external world. How will you transform that sensation? Because<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=2625'>43:45</a>: if you don't transform that sensation, you are not ever be fit to use technology. We<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=2632'>43:52</a>: shall continue to use our technology in a hostile spirit towards the external world<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=2638'>43:58</a>: and we shall wreck the external world. We're busy doing it now. There is no necessity to</details>

The speaker proposes setting up an institute of ecological technology, using places like Ceylon as an experimental ground to develop ways of production and mechanization that don't ruin the environment. [[1]](https://youtu.be/NvLlpY9vd9E#t=2646) [[2]](https://youtu.be/NvLlpY9vd9E#t=2654) [[3]](https://youtu.be/NvLlpY9vd9E#t=2663) [[4]](https://youtu.be/NvLlpY9vd9E#t=2668) [[5]](https://youtu.be/NvLlpY9vd9E#t=2675) [[6]](https://youtu.be/NvLlpY9vd9E#t=2687) [[7]](https://youtu.be/NvLlpY9vd9E#t=2691) [[8]](https://youtu.be/NvLlpY9vd9E#t=2701)This requires dedication, as one reason we make such a mess with technology is the desire for quick profits. [[9]](https://youtu.be/NvLlpY9vd9E#t=2709) [[10]](https://youtu.be/NvLlpY9vd9E#t=2719) [[11]](https://youtu.be/NvLlpY9vd9E#t=2723)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2646'>44:06</a>: abandon technology. We can't adapt. But we can certainly use it in a different spirit.<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2654'>44:14</a>: I've just been in the island of Ceylon, which is a garden. A beautiful beautiful place.<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2663'>44:23</a>: But it's completely undeveloped from a technical point of view and it's in very bad economic<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2668'>44:28</a>: circumstances because nobody wants to buy natural rubber anymore. It has no foreign<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2675'>44:35</a>: exchange. It is very peaceful. But the change has got to come. So I discussed with one of<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=2687'>44:47</a>: the high members of the government the possibility that we could set up in Ceylon an experimental<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=2691'>44:51</a>: station. Which would serve not only India eventually and Africa, but us, too. A an institute<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=2701'>45:01</a>: of ecological technology. Where we could in that experimental Island work out ways of<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=2709'>45:09</a>: production, of mechanization, automation and so on which would not ruin the island. And<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=2719'>45:19</a>: you have to do that sort of thing with a certain dedication because one of the reasons why<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=2723'>45:23</a>: we make such a mess with technology. is that the shareholders in any given corporation</details>

There's nothing wrong with being rich, but we must understand what true riches are: land, food, housing, intelligence, energy, skill, resources - not just money. [[1]](https://youtu.be/NvLlpY9vd9E#t=2730) [[2]](https://youtu.be/NvLlpY9vd9E#t=2741) [[3]](https://youtu.be/NvLlpY9vd9E#t=2745) [[4]](https://youtu.be/NvLlpY9vd9E#t=2754)Focusing solely on making money often leads to impoverishing ourselves in real terms. [[5]](https://youtu.be/NvLlpY9vd9E#t=2772) [[6]](https://youtu.be/NvLlpY9vd9E#t=2780) [[7]](https://youtu.be/NvLlpY9vd9E#t=2787) [[8]](https://youtu.be/NvLlpY9vd9E#t=2792) [[9]](https://youtu.be/NvLlpY9vd9E#t=2800)This is another example of confusing the symbol with the reality. [[10]](https://youtu.be/NvLlpY9vd9E#t=2808)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2730'>45:30</a>: want to make a fast buck. Now there's nothing wrong, you see, in wanting to be rich. There's<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2741'>45:41</a>: nothing at all wrong in being rich, in fact I think the world without rich people would<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2745'>45:45</a>: be extraordinarily boring. Point is, you have to understand what riches are. And they are<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2754'>45:54</a>: not money. Riches are land, clothes, food, housing, intelligence, energy, skill, iron,<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2772'>46:12</a>: forests, gardens- those are riches. But when you're concentrating, you see, only on making<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=2780'>46:20</a>: the buck. Doesn't occur to you that you're not really getting rich, you're just impoverishing<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=2787'>46:27</a>: yourself. Like, you know, when you're up up up up up prices, the value of the dollar goes<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=2792'>46:32</a>: down down down down down. You're just on a rat race, on a treadmill. The faster it moves,<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=2800'>46:40</a>: it doesn't get anywhere. Well it doesn't even stay in the same place. So it is that kind<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=2808'>46:48</a>: again, you see, this is another example of confusing the symbol with the reality, the</details>

They begin by discussing how we tend to see ourselves as separate from the world, with our "self" or "I" located somewhere in our head. [[1]](https://youtu.be/NvLlpY9vd9E#t=2863) [[2]](https://youtu.be/NvLlpY9vd9E#t=2864) [[3]](https://youtu.be/NvLlpY9vd9E#t=2865) This view sees the rest of our body and the external world as separate from us. [[4]](https://youtu.be/NvLlpY9vd9E#t=2866) [[5]](https://youtu.be/NvLlpY9vd9E#t=2867) [[6]](https://youtu.be/NvLlpY9vd9E#t=2868)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2863'>47:43</a>: because we say in my heart of hearts. But nowadays most of us seem to feel that whatever<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2864'>47:44</a>: it is, that I is, is located in the head. Somewhere behind the eyes and between the<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2865'>47:45</a>: ears is the center. And the rest of us is an appendage a vehicle which carries the self<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2866'>47:46</a>: around. Now popular speech also reflects the sensation. That I am very different from what<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2867'>47:47</a>: we call the other. Other people, other things. Anything that we can become aware of is sort<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=2868'>47:48</a>: of other. There is an opposition apparently between the knower and the known. And so we</details>

The speaker contrasts this with older religious views that saw humans as central to the universe and children of God. [[1]](https://youtu.be/NvLlpY9vd9E#t=2875) [[2]](https://youtu.be/NvLlpY9vd9E#t=2876) [[3]](https://youtu.be/NvLlpY9vd9E#t=2877) [[4]](https://youtu.be/NvLlpY9vd9E#t=2878) [[5]](https://youtu.be/NvLlpY9vd9E#t=2879) However, they note that modern scientific understanding has made this view implausible for many people. [[7]](https://youtu.be/NvLlpY9vd9E#t=2881) [[8]](https://youtu.be/NvLlpY9vd9E#t=2882) [[9]](https://youtu.be/NvLlpY9vd9E#t=2883) [[10]](https://youtu.be/NvLlpY9vd9E#t=2884) [[11]](https://youtu.be/NvLlpY9vd9E#t=2885)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2875'>47:55</a>: and so forth. And therefore we feel rather alone and left out of this thing. In contrast<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2876'>47:56</a>: with the ideas of Ptolemaic astronomy. Instead of being at the center of the universe. We<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2877'>47:57</a>: are on the outer limits of a minor galaxy revolving around an unimportant star. On a<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2878'>47:58</a>: small minute ball of rock. And therefore that astronomical way of looking at things is simply<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2879'>47:59</a>: overwhelming. It makes us feel not only of no importance but also very much left out.<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=2880'>48:00</a>: And as a result, that is the common sense of most people living today.<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=2881'>48:01</a>: We did of course have a religious view of our nature that we were the children of a<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=2882'>48:02</a>: loving God who is in charge of this whole operation. But very few people actually believe<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=2883'>48:03</a>: that anymore. A great many people think they ought to believe it and would like to believe<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=2884'>48:04</a>: in it but they don't. Most ministers that I know don't believe it but they feel guilty<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=2885'>48:05</a>: about this because they feel they ought to. But it became implausible. There never was</details>

They argue that our current naturalistic worldview, while rejecting supernatural explanations, has led to a negative view of nature as "blind" and "unconscious". [[1]](https://youtu.be/NvLlpY9vd9E#t=2890) [[2]](https://youtu.be/NvLlpY9vd9E#t=2891) [[3]](https://youtu.be/NvLlpY9vd9E#t=2892) [[4]](https://youtu.be/NvLlpY9vd9E#t=2893) [[5]](https://youtu.be/NvLlpY9vd9E#t=2894) [[6]](https://youtu.be/NvLlpY9vd9E#t=2895) [[7]](https://youtu.be/NvLlpY9vd9E#t=2896) [[8]](https://youtu.be/NvLlpY9vd9E#t=2897) This has resulted in a technological approach that fights against nature rather than working with it. [[13]](https://youtu.be/NvLlpY9vd9E#t=2902) [[14]](https://youtu.be/NvLlpY9vd9E#t=2903) [[15]](https://youtu.be/NvLlpY9vd9E#t=2904) [[16]](https://youtu.be/NvLlpY9vd9E#t=2905)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2890'>48:10</a>: Interestingly enough, when in the nineteenth century we switched our common sense from<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2891'>48:11</a>: supernaturalism to naturalism, one would think that a naturalist would be a person who love<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2892'>48:12</a>: nature. Just as a materialist ought to be a person who loves material but certainly<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2893'>48:13</a>: isn't. With what is called a philosophy of scientific naturalism. Naturalism is used<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2894'>48:14</a>: in a negative way. It has nothing to do with being natural it has something to do with<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=2895'>48:15</a>: being not supernatural, Merely natural. And all sorts of phrases with coined in that epoch<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=2896'>48:16</a>: which I would call put down phrases. Freud spoke of the basic psychic energy as Libido,<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=2897'>48:17</a>: which means blind lust. People like Ernst Hegel spoke of the universe as being a manifestation<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=2898'>48:18</a>: of blind energy. Think of that put down word blind. And therefore we also speak of unconscious<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=2899'>48:19</a>: mental mechanisms. And the very word unconscious as being the deeper aspect of our psyche is<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=2900'>48:20</a>: a negative word and a put down word. So is to say what you are functioning as a rational<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=2901'>48:21</a>: ego with values and with a capacity to love, is simply the epiphenomenon of a purely mechanical<br>13. <a href='https://youtu.be/NvLlpY9vd9E#t=2902'>48:22</a>: process. To bad. So as a result of this so-called naturalism we began to put up the most whopping<br>14. <a href='https://youtu.be/NvLlpY9vd9E#t=2903'>48:23</a>: fight against nature that was ever engaged in. And that fight is an expression of our<br>15. <a href='https://youtu.be/NvLlpY9vd9E#t=2904'>48:24</a>: fury and of a feeling of being left out. So that the technological experiment which became<br>16. <a href='https://youtu.be/NvLlpY9vd9E#t=2905'>48:25</a>: possible as a result of the mechanical Sciences has largely been conducted in a spirit of</details>

The speaker emphasizes the importance of ecology - understanding how all living things are interconnected and dependent on their environment. [[1]](https://youtu.be/NvLlpY9vd9E#t=2911) [[2]](https://youtu.be/NvLlpY9vd9E#t=2912) [[3]](https://youtu.be/NvLlpY9vd9E#t=2913) [[4]](https://youtu.be/NvLlpY9vd9E#t=2914) They suggest we need to change our perception of our own existence to address environmental problems effectively. [[11]](https://youtu.be/NvLlpY9vd9E#t=2921) [[12]](https://youtu.be/NvLlpY9vd9E#t=2922) [[13]](https://youtu.be/NvLlpY9vd9E#t=2923)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2911'>48:31</a>: call our ecology.<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2912'>48:32</a>: Ecology is that aspect of science which deals with the relationship between organisms and<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2913'>48:33</a>: their environments. Ecology is the study of the balance of nature. Of the way in which<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2914'>48:34</a>: every living being depends upon innumerable other living beings of all species. And also<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2915'>48:35</a>: upon inanimate forces. Air, water, temperature, gases, vegetation and all sorts of things.<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=2916'>48:36</a>: And this is one of the most important sciences that we can possibly study today. Because<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=2917'>48:37</a>: we are in a position where we realize that we cannot help interfering with the world.<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=2918'>48:38</a>: To be alive is to interfere. You must interfere. You cannot go back and say: Hands off nature.<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=2919'>48:39</a>: let's leave it all alone. Because you're stuck with it. Especially once you've started to<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=2920'>48:40</a>: interfere in a major way. We have so altered our environment that there is no hope for<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=2921'>48:41</a>: it but to go ahead. But we can, to some extent, change direction. But the only way that I<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=2922'>48:42</a>: can see of our effectively changing direction is through a transformation of the feeling<br>13. <a href='https://youtu.be/NvLlpY9vd9E#t=2923'>48:43</a>: that we have of our own existence and of what we mean by I. The reason for this is simply</details>

They then discuss how our concept of "I" or personhood is more of a symbol than a reality, comparing it to an actor's mask. [[1]](https://youtu.be/NvLlpY9vd9E#t=2933) [[2]](https://youtu.be/NvLlpY9vd9E#t=2934) [[3]](https://youtu.be/NvLlpY9vd9E#t=2935) [[4]](https://youtu.be/NvLlpY9vd9E#t=2936) The speaker argues that we are actually more like a process or "whirlpool" in the stream of nature, constantly changing but maintaining a recognizable form. [[21]](https://youtu.be/NvLlpY9vd9E#t=2953) [[22]](https://youtu.be/NvLlpY9vd9E#t=2954) [[23]](https://youtu.be/NvLlpY9vd9E#t=2955) [[24]](https://youtu.be/NvLlpY9vd9E#t=2956)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2933'>48:53</a>: of the trouble because what we think of as I is much more a symbol than it is a reality.<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2934'>48:54</a>: The living organism, the whole mind-body, is much more than anything we mean by I. I<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2935'>48:55</a>: largely stands for your personality. Your role in life. And the very word person, as<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2936'>48:56</a>: you probably know, comes from the Latin persona. A word originally used for the mask worn by<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2937'>48:57</a>: actors in Graeco-Roman drama. That through which sound comes, because the mask had a<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=2938'>48:58</a>: mega phonic mouth to carry the voice an open air theaters. So when you speak of being a<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=2939'>48:59</a>: real person, it really means being a genuine fake. Because the personality is only the<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=2940'>49:00</a>: front. What is behind it? Well of course the organism is behind it. The whole organism.<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=2941'>49:01</a>: And we must be very careful not to confuse the organism with various symbols that we<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=2942'>49:02</a>: have for it, because those symbols can be extremely misleading. If we say the organism<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=2943'>49:03</a>: is the body, what we usually mean by the body, is an impoverished meaning. When we speak<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=2944'>49:04</a>: of my body that is to say my vehicle, my physical automobile. That is an unenriched meaning<br>13. <a href='https://youtu.be/NvLlpY9vd9E#t=2945'>49:05</a>: of the word body. Because what you really are as a body, as a living organism, is not<br>14. <a href='https://youtu.be/NvLlpY9vd9E#t=2946'>49:06</a>: some sort of separate existence coated by a skin which divides you from the rest of<br>15. <a href='https://youtu.be/NvLlpY9vd9E#t=2947'>49:07</a>: the world. Shakespeare has King John saying to Hubert: "Within this wall of flesh there<br>16. <a href='https://youtu.be/NvLlpY9vd9E#t=2948'>49:08</a>: is a soul counts thee her creditor." Within this wall of flesh: the skin considered as<br>17. <a href='https://youtu.be/NvLlpY9vd9E#t=2949'>49:09</a>: a barrier. When actually, from a biological point of view, the human skin and all skins<br>18. <a href='https://youtu.be/NvLlpY9vd9E#t=2950'>49:10</a>: are osmotic membranes. You know when you get something by osmosis. By sort of soaking it<br>19. <a href='https://youtu.be/NvLlpY9vd9E#t=2951'>49:11</a>: in. So in the same way one's skin is a spongy construction full of holes. Full of communicators,<br>20. <a href='https://youtu.be/NvLlpY9vd9E#t=2952'>49:12</a>: nerve endings. And your skin is simply a vibrating membrane through which the so-called external<br>21. <a href='https://youtu.be/NvLlpY9vd9E#t=2953'>49:13</a>: world flows into you and through you. So that you yourself actually are not so much an entity<br>22. <a href='https://youtu.be/NvLlpY9vd9E#t=2954'>49:14</a>: that moves around in an environment. You are much more like a whirlpool in a stream. And,<br>23. <a href='https://youtu.be/NvLlpY9vd9E#t=2955'>49:15</a>: as you know, the whirlpool is constant only in its doing, that is to say in its whirling.<br>24. <a href='https://youtu.be/NvLlpY9vd9E#t=2956'>49:16</a>: And you could recognize individual whirligigs in a stream. But the water is flowing through</details>

Consider, for example, a university. [[1]](https://youtu.be/NvLlpY9vd9E#t=2964) The undergraduate student body changes every four years. The faculty changes periodically. The buildings keep changing. [[2]](https://youtu.be/NvLlpY9vd9E#t=2965) What constitutes the University of California? It certainly isn't the faculty, it isn't the students, it isn't the governors, it isn't the administration, it isn't the buildings. [[3]](https://youtu.be/NvLlpY9vd9E#t=2966) [[4]](https://youtu.be/NvLlpY9vd9E#t=2967) What is it? It's a doing, a behavior, a university-ing process of study and experiment. [[5]](https://youtu.be/NvLlpY9vd9E#t=2968) So it is exactly the same with you. You flow. You are a process. [[6]](https://youtu.be/NvLlpY9vd9E#t=2969)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2964'>49:24</a>: is changing constantly. Imagine for example a university. The student body undergraduate<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2965'>49:25</a>: changes every four years. The faculty changes every so often. The buildings keep changing<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2966'>49:26</a>: more and more. What constitutes the University of California? It certainly isn't the faculty,<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2967'>49:27</a>: it isn't the students, it isn't the governors, it isn't the administration, it isn't the<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2968'>49:28</a>: buildings. What is it? Why a doing. A behavior. A university-ing process of study and experiment<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=2969'>49:29</a>: and so on. So it is exactly the same with you. You flow. You are a process.</details>

But how do we draw the line about this process and its relationship to all other processes? [[1]](https://youtu.be/NvLlpY9vd9E#t=2970) We find that a very difficult thing to do the more we think about it. [[2]](https://youtu.be/NvLlpY9vd9E#t=2971) If you really felt with your whole organism, instead of just with that part called conscious attention, you would become aware of this flowing fact. [[3]](https://youtu.be/NvLlpY9vd9E#t=2972) You would get a very strange feeling which at first might frighten you. [[4]](https://youtu.be/NvLlpY9vd9E#t=2973) It is possible to have this feeling. [[5]](https://youtu.be/NvLlpY9vd9E#t=2974) You might not be quite sure how to interpret it. [[6]](https://youtu.be/NvLlpY9vd9E#t=2975) You might feel that you yourself were doing everything else that's happening. That would be one way of feeling it. [[7]](https://youtu.be/NvLlpY9vd9E#t=2976) The other way would be that you are doing nothing at all, but that everything else is doing you. [[8]](https://youtu.be/NvLlpY9vd9E#t=2977) You would feel completely passive, like a puppet on strings. [[9]](https://youtu.be/NvLlpY9vd9E#t=2978) On the other hand, if you got the feeling that you were doing it all, you would feel like God Almighty. [[10]](https://youtu.be/NvLlpY9vd9E#t=2979) It is very easy for our consciousness to slip into this state of sensation. [[11]](https://youtu.be/NvLlpY9vd9E#t=2980) It can happen spontaneously, like measles. [[12]](https://youtu.be/NvLlpY9vd9E#t=2981) It can happen through training, as when someone practices yoga. It can happen chemically when certain drugs are taken. [[13]](https://youtu.be/NvLlpY9vd9E#t=2982)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2970'>49:30</a>: But how do we draw the line about this process and its relationship to all other processes?<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2971'>49:31</a>: We find that a very difficult thing to do the more you think about it. If you really<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2972'>49:32</a>: felt with your whole organism, instead of just with that part of it called conscious<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2973'>49:33</a>: attention, you would become aware of this flowing fact. And you would get a very strange<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2974'>49:34</a>: feeling which at first might frighten you. It is possible of course to have this feeling.<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=2975'>49:35</a>: And the feeling is like this: You would not be quite sure how to interpret it. You might<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=2976'>49:36</a>: feel that you yourself were doing everything else that's happening. That would be one way<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=2977'>49:37</a>: of feeling. The other way of feeling it would be that you are doing nothing at all. But<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=2978'>49:38</a>: that everything else is doing you. And you would feel completely passive like a puppet<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=2979'>49:39</a>: on the end of strings. Although on the other hand if you got the feeling that you were<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=2980'>49:40</a>: doing it all you would feel like God Almighty. It is very easy for our consciousness to slip<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=2981'>49:41</a>: into this state of sensation. It can happen spontaneously, like measles. It can happen<br>13. <a href='https://youtu.be/NvLlpY9vd9E#t=2982'>49:42</a>: by training. As when someone practices yoga. It can happen chemically as when certain drugs</details>

One has to be very careful about this feeling because it's enormously easy to misinterpret. [[1]](https://youtu.be/NvLlpY9vd9E#t=2983) Either as being omnipotent, being God in the personal, literal sense, [[2]](https://youtu.be/NvLlpY9vd9E#t=2984) or as being helpless and merely driven. What should be understood is that both these ways of feeling are right. [[3]](https://youtu.be/NvLlpY9vd9E#t=2985) Only they must both be taken together. To be simultaneously omnipotent and helpless. [[4]](https://youtu.be/NvLlpY9vd9E#t=2986) These are two poles, opposite poles of one and the same state. [[5]](https://youtu.be/NvLlpY9vd9E#t=2987)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2983'>49:43</a>: are taken. And one has to be very careful about this feeling because it's enormously<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2984'>49:44</a>: easy to misinterpret. Either as being omnipotent, being God in the personal, literal sense.<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2985'>49:45</a>: Or as being helpless and merely driven. Now what should be understood is that both these<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2986'>49:46</a>: ways of feeling are right. Only they must both be taken together. To be simultaneously<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2987'>49:47</a>: omnipotent and helpless. These are two poles, opposite poles of one and the same state.</details>

The message that is coming through, which we find difficult to understand because it's contrary to our common sense and our whole history and conditioning, [[1]](https://youtu.be/NvLlpY9vd9E#t=2988) [[2]](https://youtu.be/NvLlpY9vd9E#t=2989) is this: you as a living organism and all that is going on in your environment constitute a single process. [[3]](https://youtu.be/NvLlpY9vd9E#t=2990) In physics, we would call this a unified field. [[4]](https://youtu.be/NvLlpY9vd9E#t=2991) It's a single process like a pattern, but you know any pattern has all sorts of subsidiary wiggles in it. [[5]](https://youtu.be/NvLlpY9vd9E#t=2992) Like the organism itself is a unified pattern but it's full of wiggles. [[6]](https://youtu.be/NvLlpY9vd9E#t=2993) All sorts of tubes and organs and bones and nerves and so on, working in this way. [[7]](https://youtu.be/NvLlpY9vd9E#t=2994)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2988'>49:48</a>: Because the message that is coming through, and that we find difficulty in understanding,<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2989'>49:49</a>: because it's contrary to our common sense, contrary to our whole history and conditioning,<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2990'>49:50</a>: the message that's coming through is: you as a living organism and all that is going<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2991'>49:51</a>: on in your environment constitute a single process. What in physics we would call a unified<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2992'>49:52</a>: field. A single process like a pattern but you know any pattern has all sorts of subsidiary<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=2993'>49:53</a>: wiggles in it. Like the organism itself is a unified pattern but it's full of wiggles.<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=2994'>49:54</a>: All sorts of tubes and organs and bones and nerves and so on. Working in this way. You</details>

You know, the body doesn't have a boss. We could pitch a big argument: Who is really the top dog in your body, your stomach or your brain? [[1]](https://youtu.be/NvLlpY9vd9E#t=2995) I can argue both ways. [[2]](https://youtu.be/NvLlpY9vd9E#t=2996) Let's first argue for the stomach. The stomach is fundamental. That's what eats, and eating is the fundamental thing of being alive. [[3]](https://youtu.be/NvLlpY9vd9E#t=2997) By putting food into the stomach, it digests it and from there it goes out and energizes everything else. [[4]](https://youtu.be/NvLlpY9vd9E#t=2998) Obviously, the stomach is the most important. The hands, the mouth, the feet all exist to serve the stomach, [[5]](https://youtu.be/NvLlpY9vd9E#t=2999) and naturally, as a final achievement of the stomach, is the brain. [[6]](https://youtu.be/NvLlpY9vd9E#t=3000) Evolving later in the evolutionary process as a gadget up there to scavenge around and find stuff for the stomach to eat, that's the function of the brain. [[7]](https://youtu.be/NvLlpY9vd9E#t=3001) [[8]](https://youtu.be/NvLlpY9vd9E#t=3002)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=2995'>49:55</a>: know the body doesn't have a boss. We could pitch a big argument. Who is really the top<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=2996'>49:56</a>: dog in your body, your stomach or your brain? I can argue for both ways. Let's first argue<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=2997'>49:57</a>: for the stomach. The stomach is fundamental. That's what eats and eating is the fundamental<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=2998'>49:58</a>: thing of being alive. By putting food into the stomach it digests it and from there it<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=2999'>49:59</a>: goes out and energizes everything else. Obviously the stomach is the most important. The hands,<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3000'>50:00</a>: the mouth, the feet all exist to serve the stomach and naturally as a final achievement<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3001'>50:01</a>: of the stomach is the brain. Evolving later in the evolutionary process as a gadget up<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=3002'>50:02</a>: there to scavenge around and find stuff for the stomach to eat, that's the function the</details>

But now let's take the argument to the side of the brain. The brain says, "Oh no, no. Just because I arrived late doesn't mean that I'm unimportant. I was being gotten ready for. [[1]](https://youtu.be/NvLlpY9vd9E#t=3003) [[2]](https://youtu.be/NvLlpY9vd9E#t=3004) Because I am the thing that is the flower at the top of this thing. And this tube with stomachs in it and things below was preparing for me and the stomach is my servant. [[3]](https://youtu.be/NvLlpY9vd9E#t=3005) [[4]](https://youtu.be/NvLlpY9vd9E#t=3006) It is doing all the dirty work and getting energy to put currents through my wonderful circuits. So that by the creation of all the goods of the mind of the Arts and Sciences and religion and philosophy and so on, I shall be the true head of man." [[5]](https://youtu.be/NvLlpY9vd9E#t=3007) [[6]](https://youtu.be/NvLlpY9vd9E#t=3008)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3003'>50:03</a>: brain. But now let's take the argument to the side of the brain. The brain says: "Oh<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3004'>50:04</a>: no no. I'm not. Just because I arrived late doesn't mean that I'm unimportant. I was being<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3005'>50:05</a>: gotten ready for. Because I am the thing that is the flower at the top of this thing. And<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3006'>50:06</a>: this tube with stomachs in it and things below was preparing for me and the stomach is my<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3007'>50:07</a>: servant. It is doing all the dirty work and getting energy to put currents through my<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3008'>50:08</a>: wonderful circuits. So that by the creation of all the goods of the mind of the Arts and</details>

Well, both arguments are right. Because you have a relationship between stomach and brain, which is a sort of polarity. [[1]](https://youtu.be/NvLlpY9vd9E#t=3009) The one exists for the other. It's like when you prop up two sticks against each other, they will stand up so long as they lean on each other. [[2]](https://youtu.be/NvLlpY9vd9E#t=3010) [[3]](https://youtu.be/NvLlpY9vd9E#t=3011) Take one away and the other collapses. So chop off the head and the stomach is finished. Take out the stomach and the head is finished. [[4]](https://youtu.be/NvLlpY9vd9E#t=3012) [[5]](https://youtu.be/NvLlpY9vd9E#t=3013)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3009'>50:09</a>: Sciences and religion and philosophy and so on I shall be the true head of man. Well both<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3010'>50:10</a>: arguments are right. Because you have a relationship between stomach and brain, which is a sort<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3011'>50:11</a>: of polarity. The one exists for the other. It's like when you prop up two sticks against<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3012'>50:12</a>: each other they will stand up so long as they lean on each other. Take one away and the<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3013'>50:13</a>: other collapses. So chop off the head and the stomach is finished. Take out the stomach</details>

This is the way all organic life proceeds. It's different with mechanical life. [[1]](https://youtu.be/NvLlpY9vd9E#t=3014) Because the mechanism must invariably have a boss. The man who puts the machine together, the person who constructs the computer, who designs it, who asks it questions, who programs it. [[2]](https://youtu.be/NvLlpY9vd9E#t=3015) [[3]](https://youtu.be/NvLlpY9vd9E#t=3016) He is the boss. But organisms don't have bosses. They are essentially, I would say, democratic arrangements. [[4]](https://youtu.be/NvLlpY9vd9E#t=3017) Where, somehow, in a marvelous way, an enormous company of cells are working together. [[5]](https://youtu.be/NvLlpY9vd9E#t=3018) But that isn't the way the body was formed, sort of one day a lot of cells all crept together and said: We are a body. [[6]](https://youtu.be/NvLlpY9vd9E#t=3019) [[7]](https://youtu.be/NvLlpY9vd9E#t=3020)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3014'>50:14</a>: and the head is finished. So this is the way all organic life proceeds. It's different<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3015'>50:15</a>: with mechanical life. Because the mechanism must invariably have a boss. The man who puts<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3016'>50:16</a>: the machine together, the person who constructs the computer, who designs it, who asks it<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3017'>50:17</a>: questions, who programs it. He is the boss. But organisms don't have bosses. They are<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3018'>50:18</a>: essentially, I would say, democratic arrangements. Where, somehow, in a marvelous way, an enormous<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3019'>50:19</a>: company of cells are working together. But that isn't the way, the body wasn't sort of:<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3020'>50:20</a>: one day a lot of cells all crept to together and said: We are a body. That does sometimes</details>

That does sometimes happen in the biological domain, but much rather this: when you watch the gestation of a mammal, you see first of all a very simple little organism. [[1]](https://youtu.be/NvLlpY9vd9E#t=3021) Which swells. And as it swells, it becomes more and more complicated from within. [[2]](https://youtu.be/NvLlpY9vd9E#t=3022) No parts are added, nothing is screwed on, there's no welding done or anything like that. It bulges and of course it does absorb material, but it transforms it. [[3]](https://youtu.be/NvLlpY9vd9E#t=3023) [[4]](https://youtu.be/NvLlpY9vd9E#t=3024) But all of it works together at once like the legs of a centipede. Like you work all together at once. [[5]](https://youtu.be/NvLlpY9vd9E#t=3025) [[6]](https://youtu.be/NvLlpY9vd9E#t=3026)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3021'>50:21</a>: happen in the biological domain, but much rather this: when you watch the gestation<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3022'>50:22</a>: of a mammal, you see first of all a very simple little organism. Which swells. And as it swells,<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3023'>50:23</a>: it becomes more and more complicated from within. No parts are added, nothing is screwed<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3024'>50:24</a>: on, there's no welding done or anything like that. It bulges and of course it does absorb<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3025'>50:25</a>: material, but it transforms it. But all of it works together at once like the legs of<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3026'>50:26</a>: a centipede. Like you work all together at once. For, you see, when we come down to it,</details>

For, you see, when we come down to it, you think you decide things, but you don't know how you do it. [[1]](https://youtu.be/NvLlpY9vd9E#t=3027) How do you open and close your hand? You can decide "I will now open my hand" and do it, but you don't know how it's done. [[2]](https://youtu.be/NvLlpY9vd9E#t=3028) And yet in a sense you do know how it's done because you say "I know how to open my hand." [[3]](https://youtu.be/NvLlpY9vd9E#t=3029) But you don't know in words. You can't explain it. Still less can you explain how you see, still less can you explain how you are conscious. [[4]](https://youtu.be/NvLlpY9vd9E#t=3030) [[5]](https://youtu.be/NvLlpY9vd9E#t=3031) How are you an ego? Well, you don't know because the springs of being conscious, of being an ego are outside the surveillance of consciousness. [[6]](https://youtu.be/NvLlpY9vd9E#t=3032) They're somehow underneath. And that lets the cat out of the bag at once, because you see that what is "I" is something very, very much deeper than the superficial consciousness. [[7]](https://youtu.be/NvLlpY9vd9E#t=3033) [[8]](https://youtu.be/NvLlpY9vd9E#t=3034) And what you call "I" in the sense of the voluntary willing center, ego, has very little to do with it. [[9]](https://youtu.be/NvLlpY9vd9E#t=3035) You are just a watchman on top of the mast or a radar on a ship that is scanning the environment by conscious attention looking out for trouble. Or looking for food. [[10]](https://youtu.be/NvLlpY9vd9E#t=3036) The real you is much too complicated to think about. [[11]](https://youtu.be/NvLlpY9vd9E#t=3037)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3027'>50:27</a>: you think you decide things, but you don't know how you do it.<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3028'>50:28</a>: How do you open and close your hand? And you can decide I will now open my and do it, but<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3029'>50:29</a>: you don't know how it's done. And yet in a sense you do know how it's done because you<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3030'>50:30</a>: say I know how to open my hand. But you don't know in words. You can't explain it. Still<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3031'>50:31</a>: less can you explain how you see, still less can you explain how you are conscious. How<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3032'>50:32</a>: are you an ego? Well you don't know because the springs of being conscious, of being an<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3033'>50:33</a>: ego are outside the surveillance of consciousness. They're somehow underneath. And that lets<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=3034'>50:34</a>: the cat out of the bag at once, because you see that what is I is something very very<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=3035'>50:35</a>: much deeper than the superficial consciousness. And what you call I in the sense of the voluntary<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=3036'>50:36</a>: willing center, ego, has very little to do with it. You are just a watchman on top of<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=3037'>50:37</a>: the mast or a radar on a ship that is scanning the environment by conscious attention looking</details>

Supposing when you woke up in the morning, you had to switch yourself on. [[1]](https://youtu.be/NvLlpY9vd9E#t=3038) That is to say, you had to, by an act of conscious attention, go through your brain and turn on all the synapses necessary for wakeful life. [[2]](https://youtu.be/NvLlpY9vd9E#t=3039) [[3]](https://youtu.be/NvLlpY9vd9E#t=3040) It'd take you hours. Supposing you really had to be conscious of all the details involved in walking or in breathing or in circulation of the blood. [[4]](https://youtu.be/NvLlpY9vd9E#t=3041) [[5]](https://youtu.be/NvLlpY9vd9E#t=3042) You'd never get around to it. So you see when we inspect the physical world with conscious attention the first thing that strikes us is that the physical world is extraordinarily complicated. [[6]](https://youtu.be/NvLlpY9vd9E#t=3043) [[7]](https://youtu.be/NvLlpY9vd9E#t=3044) How can it possibly be organized? But actually the physical world is not complicated at all. [[8]](https://youtu.be/NvLlpY9vd9E#t=3045) What is complicated is the task of trying to describe it in words. Or of trying to figure it out in numbers. [[9]](https://youtu.be/NvLlpY9vd9E#t=3046) Because that is analogous to the task of, say, removing the water from the Pacific Ocean into the Atlantic with a beer mug. [[10]](https://youtu.be/NvLlpY9vd9E#t=3047) We can only take one mug at a time. And so we say in popular speech you can only think of one thing at a time. [[11]](https://youtu.be/NvLlpY9vd9E#t=3048) [[12]](https://youtu.be/NvLlpY9vd9E#t=3049)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3038'>50:38</a>: out for trouble. Or looking for food. The real you is much too complicated to think<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3039'>50:39</a>: about. Supposing when you woke up in the morning, you had to switch yourself on. That is to<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3040'>50:40</a>: say, you had to, by an act of conscious attention, to go through your brain and turn on all the<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3041'>50:41</a>: synapses necessary for wakeful life. It'd take you hours. Supposing you really had to<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3042'>50:42</a>: be conscious of all the details involved in walking or in breathing or in circulation<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3043'>50:43</a>: of the blood. You'd never get around to it. So you see when we inspect the physical world<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3044'>50:44</a>: with conscious attention the first thing that strikes us is that the physical world is extraordinarily<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=3045'>50:45</a>: complicated. How can it possibly be organized. But actually the physical world is not complicated<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=3046'>50:46</a>: at all. What is complicated is the task of trying to describe it in words. Or of trying<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=3047'>50:47</a>: to figure it out in numbers. Because that is analogous to the task of, say, removing<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=3048'>50:48</a>: the water from the Pacific Ocean into the Atlantic with a beer mug. We can only take<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=3049'>50:49</a>: one mug at a time. And so we say in popular speech you can only think of one thing at</details>

That's not exactly true but what it reveals is that thinking, that conscious thought is a kind of calculus in which we understand things bit by bit. [[1]](https://youtu.be/NvLlpY9vd9E#t=3050) And it leads us into the superstition that things really are bits. [[2]](https://youtu.be/NvLlpY9vd9E#t=3051) Now, when you eat chicken of course you have to bite it. And you take it in bits. [[3]](https://youtu.be/NvLlpY9vd9E#t=3052) And to make it easier to bite you order from the grocer a cut up fryer. [[4]](https://youtu.be/NvLlpY9vd9E#t=3053) But you don't get cut up fryers out of eggs. Because you see although we can speak of A chicken, AN egg, or A body, it is not actually a bit. [[5]](https://youtu.be/NvLlpY9vd9E#t=3054) [[6]](https://youtu.be/NvLlpY9vd9E#t=3055) It hasn't been bitten off except for purposes of thinking. [[7]](https://youtu.be/NvLlpY9vd9E#t=3056)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3050'>50:50</a>: a time. That's not exactly true but what it reveals is that thinking, that conscious thought<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3051'>50:51</a>: is a kind of calculus in which we understand things bit by bit. And it leads us into the<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3052'>50:52</a>: superstition that things really are bits. Now, when you eat chicken of course you have<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3053'>50:53</a>: to bite it. And you take it in bits. And to make it easier to bite you order from the<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3054'>50:54</a>: grocer a cut up fryer. But you don't get cut up fryers out of eggs. Because you see although<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3055'>50:55</a>: we can speak of A chicken, AN egg, or A body, it is not actually a bit. It hasn't been bitten<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3056'>50:56</a>: off except for purposes of thinking. Now.</details>

This is beautifully brought out in a passage from Whitehead, which I will read to you. [[1]](https://youtu.be/NvLlpY9vd9E#t=3057) He is discussing the nineteenth century philosophy of science, which I was just discussing, too. [[2]](https://youtu.be/NvLlpY9vd9E#t=3058) And He's saying in this philosophy: All our impressions of nature are simply products of our minds. [[3]](https://youtu.be/NvLlpY9vd9E#t=3059) Nature gets credit which should in truth be reserved for ourselves. The rose for its scent, the nightingale for his song and the sun for his radiance. [[4]](https://youtu.be/NvLlpY9vd9E#t=3060) [[5]](https://youtu.be/NvLlpY9vd9E#t=3061) The poets are entirely mistaken. They should address their lyrics to themselves and should turn them into odes of self congratulation on the excellency of the human mind. [[6]](https://youtu.be/NvLlpY9vd9E#t=3062) Nature is a dull affair. Soundless, scentless, colorless. Merely the hurrying of material, endlessly, meaninglessly. [[7]](https://youtu.be/NvLlpY9vd9E#t=3063) [[8]](https://youtu.be/NvLlpY9vd9E#t=3064)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3057'>50:57</a>: This is beautifully brought out in a passage from Whitehead, which I will read to you.<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3058'>50:58</a>: He is discussing the nineteenth century philosophy of science, which I was just discussing, too.<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3059'>50:59</a>: And He's saying in this philosophy: All our impressions of nature are simply products<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3060'>51:00</a>: of our minds. Nature gets credit which should in truth be reserved for ourselves. The rose<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3061'>51:01</a>: for its sent the nightingale for his song and the sun for his radiance. The poets are<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3062'>51:02</a>: entirely mistaken. They should address their lyrics to themselves and should turn them<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3063'>51:03</a>: into odes of self congratulation on the excellency of the human mind. Nature is a dull affair.<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=3064'>51:04</a>: Soundless, scentless, colorless. Merely the hurrying of material, endlessly, meaninglessly.</details>

However you disguise it, this is the practical outcome of the characteristic scientific philosophy which closed the seventeenth century. [[1]](https://youtu.be/NvLlpY9vd9E#t=3065) In the first place we must note its astounding efficiency as a system of concepts for the organization of scientific research. [[2]](https://youtu.be/NvLlpY9vd9E#t=3066) In this respect, it is fully worthy of the genius of the century which produced it. It has held its own as the guiding principle of scientific studies ever since. [[3]](https://youtu.be/NvLlpY9vd9E#t=3067) It is still reigning. Every university in the world organizes itself in accordance with it. [[4]](https://youtu.be/NvLlpY9vd9E#t=3068) [[5]](https://youtu.be/NvLlpY9vd9E#t=3069) No alternative system of organizing the pursuit of scientific truth has been suggested. [[6]](https://youtu.be/NvLlpY9vd9E#t=3070) It is not only reigning, but it is without a rival. And yet it is quite unbelievable. [[7]](https://youtu.be/NvLlpY9vd9E#t=3071) This conception of the universe is surely framed in terms of high abstractions and the paradox only arises because we have mistaken our abstractions for concrete realities. [[8]](https://youtu.be/NvLlpY9vd9E#t=3072) He calls that, you see, the fallacy of misplaced concreteness. [[9]](https://youtu.be/NvLlpY9vd9E#t=3073) The attribution, in other words, of our bitty way of thinking to the world which we are biting. That is misplaced concreteness. [[10]](https://youtu.be/NvLlpY9vd9E#t=3074) [[11]](https://youtu.be/NvLlpY9vd9E#t=3075)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3065'>51:05</a>: However you disguise it, this is the practical outcome of the characteristic scientific philosophy<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3066'>51:06</a>: which closed even the seventeenth century. In the first place we must note its astounding<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3067'>51:07</a>: efficiency as a system of concepts for the organization of scientific research. In this<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3068'>51:08</a>: respect, it is fully worthy of the genius of the century which produced it. It has held<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3069'>51:09</a>: its own as the guiding principle of scientific studies ever since. It is still raining. Every<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3070'>51:10</a>: university in the world organizes itself in accordance with it. No alternative system<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3071'>51:11</a>: of organizing the pursuit of scientific truth has been suggested. It is not only reigning,<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=3072'>51:12</a>: but it is without a rival. And yet it is quite unbelievable. This conception of the universe<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=3073'>51:13</a>: is surely framed in terms of high abstractions and the paradox only arises because we have<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=3074'>51:14</a>: mistaken our abstractions for concrete realities. He calls that, you see, the fallacy of misplaced<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=3075'>51:15</a>: concreteness. The attribution, in other words, to our bitty way of thinking to the world</details>

The separations between things are abstract. They are concepts. [[1]](https://youtu.be/NvLlpY9vd9E#t=3076) In the same way, for example, as lines of latitude and longitude are concepts. [[2]](https://youtu.be/NvLlpY9vd9E#t=3077) Even though a Russian poet has recently made a beautiful poem about the world being like a ball carried in a net bag. [[3]](https://youtu.be/NvLlpY9vd9E#t=3078) But you never expect to trip over the equator when you cross it. [[4]](https://youtu.be/NvLlpY9vd9E#t=3079) And although it is something quite abstract and does not exist in nature, it's extremely useful for purposes of navigation. [[5]](https://youtu.be/NvLlpY9vd9E#t=3080) So in the same way, biting and having words to describe particular events or particular wiggles in the universal pattern are very useful. [[6]](https://youtu.be/NvLlpY9vd9E#t=3081) But they are very dangerous when you confuse things with natural events. [[7]](https://youtu.be/NvLlpY9vd9E#t=3082) Because then you get into this sort of trouble. [[8]](https://youtu.be/NvLlpY9vd9E#t=3083) [[9]](https://youtu.be/NvLlpY9vd9E#t=3084)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3076'>51:16</a>: which we are biting. That is misplaced concreteness. The separations between things are abstract.<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3077'>51:17</a>: They are concepts. In the same way, for example, as lines of latitude and longitude are concepts.<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3078'>51:18</a>: Even though a Russian poet has recently made a beautiful poem about the world being like<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3079'>51:19</a>: a ball carried in a net bag. But you never expect to trip over the equator when you cross<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3080'>51:20</a>: it. And although it is something quite abstract and does not exist in nature, it's extremely<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3081'>51:21</a>: useful for purposes of navigation. So in the same way, bit-ing and having words to describe<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3082'>51:22</a>: particular events or particular wiggles in the universal pattern are very useful. But<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=3083'>51:23</a>: they are very dangerous when you confuse things with natural events. Because then you get<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=3084'>51:24</a>: into this sort of trouble.</details>

The trouble of the sorcerer. The surgeon who is too much of a specialist in one organ runs into that organ and alters it and does what he considers a better mechanical job than the Lord did. [[1]](https://youtu.be/NvLlpY9vd9E#t=3085) [[2]](https://youtu.be/NvLlpY9vd9E#t=3086) But then he discovers to his dismay and the even greater dismay of his patient that the operation has unforeseen consequences in some other part of the organism. [[3]](https://youtu.be/NvLlpY9vd9E#t=3087) Because he didn't realize the connection. When, likewise, we object to certain insect pests. Oh, we say, get rid of them. D.D.T.. [[4]](https://youtu.be/NvLlpY9vd9E#t=3088) [[5]](https://youtu.be/NvLlpY9vd9E#t=3089) So. Then we found we got rid of something else we didn't want to get rid of. [[6]](https://youtu.be/NvLlpY9vd9E#t=3090) And worse still that this insect that we didn't like was doing a job for us in some manner of which we weren't aware and we only become aware of it when suddenly we find ourselves covered with another kind of fly altogether, or with some sort of bacteria which this insect kept down. [[7]](https://youtu.be/NvLlpY9vd9E#t=3091) [[8]](https://youtu.be/NvLlpY9vd9E#t=3092) [[9]](https://youtu.be/NvLlpY9vd9E#t=3093) Watch it, watch it, watch it because nature does not consist of separate things which you can just pull out like parts from an engine. [[10]](https://youtu.be/NvLlpY9vd9E#t=3094) [[11]](https://youtu.be/NvLlpY9vd9E#t=3095)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3085'>51:25</a>: The trouble of the sorcerer. The surgeon who is too much of a specialist in one organ runs<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3086'>51:26</a>: into that organ and alters it and does what he considers a better mechanical job than<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3087'>51:27</a>: the Lord did. But then he discovers to his dismay and the even greater dismay of his<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3088'>51:28</a>: patient that the operation has unforeseen consequences in some other part of the organism.<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3089'>51:29</a>: Because he didn't realize the connection. When, likewise, we object to certain insect<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3090'>51:30</a>: pests. Oh, we say, get rid of them. D.D.T.. So. Then we found we got rid of something<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3091'>51:31</a>: else we didn't want to get rid of. And worse still that this insect that we didn't like<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=3092'>51:32</a>: was doing a job for us in some manner of which weren't aware and we only become aware of<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=3093'>51:33</a>: it when suddenly we find ourselves covered with another kind of fly altogether, or with<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=3094'>51:34</a>: some sort of bacteria which this insect kept down. Watch it, watch it, watch it because<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=3095'>51:35</a>: nature does not consist of separate things which you can just pull out like parts from</details>

Take the case of bees and flowers. Oh, we always use the bees and flowers to explain fundamental things about life but we're going to go deeper than sex this time. [[1]](https://youtu.be/NvLlpY9vd9E#t=3096) [[2]](https://youtu.be/NvLlpY9vd9E#t=3097) Fascinating thing about bees and flowers is they are very different looking things. A flower sits still and blooms and it smells, or stinks to be correct. [[3]](https://youtu.be/NvLlpY9vd9E#t=3098) The bee moves about and buzzes. But they are all one organism. [[4]](https://youtu.be/NvLlpY9vd9E#t=3099) You don't find flowers without bees, you don't find bees without flowers. They are just as much one as your head and your feet which also look very different. [[5]](https://youtu.be/NvLlpY9vd9E#t=3100) [[6]](https://youtu.be/NvLlpY9vd9E#t=3101) So in that sense you see we are one with the incredible complexity of processes and wiggles upon which we depend. [[7]](https://youtu.be/NvLlpY9vd9E#t=3102) Although to say "upon which we depend" is not quite accurate. Because that separates us from it as if I were hanging onto a beam and depending on it. [[8]](https://youtu.be/NvLlpY9vd9E#t=3103) It isn't like that. You don't depend on it, because it depends on you. It's a mutual arrangement. [[9]](https://youtu.be/NvLlpY9vd9E#t=3104) [[10]](https://youtu.be/NvLlpY9vd9E#t=3105) And it isn't that one bit of this sort of came first. Although that sometimes happens. But it's always there in potentiality, what came later. [[11]](https://youtu.be/NvLlpY9vd9E#t=3106) [[12]](https://youtu.be/NvLlpY9vd9E#t=3107)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3096'>51:36</a>: an engine. Take the case of bees and flowers. Oh, we always use the bees and flowers to<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3097'>51:37</a>: explain fundamental things about life but we're going to go deeper than sex this time.<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3098'>51:38</a>: Fascinating thing about bees and flowers is they are very different looking things. A<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3099'>51:39</a>: flower sits still and blooms and it smells, or stinks to be correct. The bee moves about<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3100'>51:40</a>: and buzzes. But they are all one organism. You don't find flowers without bees, you don't<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3101'>51:41</a>: find bees without flowers. They are just as much one as your head and your feet which<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3102'>51:42</a>: also look very different. So in that sense you see we are one with the incredible complexity<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=3103'>51:43</a>: of processes and wiggles upon which we depend. Although to say "upon which we depend" is<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=3104'>51:44</a>: not quite accurate. Because that separates us from it as if I were hanging onto a beam<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=3105'>51:45</a>: and depending on it. It isn't like that. You don't depend on it, because it depends on<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=3106'>51:46</a>: you. It's a mutual arrangement. And it isn't that one bit of this sort of came first. Although<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=3107'>51:47</a>: that sometimes happens. But it's always there in potentiality, what came later. But it's</details>

But it's rather in the same way that, when a flower opens, you see all the different petals extending simultaneously. [[1]](https://youtu.be/NvLlpY9vd9E#t=3108) Especially when you watch a fast motion movie of a flower opening. And so in the same way there is a simultaneous arrival, or evolution, of the human organism and the human environment and thus biologists speak about the evolution of an environment as well as the evolution of an organism in it. [[2]](https://youtu.be/NvLlpY9vd9E#t=3109) [[3]](https://youtu.be/NvLlpY9vd9E#t=3110) [[4]](https://youtu.be/NvLlpY9vd9E#t=3111) In other words, human beings could not have appeared on this planet until its temperature had lowered to a certain degree, until the atmosphere contained certain gases as a result of vegetative development. [[5]](https://youtu.be/NvLlpY9vd9E#t=3112) And then the environment became evolved enough for human beings to appear in it. [[6]](https://youtu.be/NvLlpY9vd9E#t=3113) [[7]](https://youtu.be/NvLlpY9vd9E#t=3114) Evolved enough? I'll say something further: intelligent enough for the appearance in it of intelligent beings. [[8]](https://youtu.be/NvLlpY9vd9E#t=3115) [[9]](https://youtu.be/NvLlpY9vd9E#t=3116)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3108'>51:48</a>: rather in the same way that, when a flower opens, you see all the different petals extending<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3109'>51:49</a>: simultaneously. Especially when you watch a fast motion movie of a flower opening. And<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3110'>51:50</a>: so in the same way there is a simultaneous arrival, or evolution, of the human organism<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3111'>51:51</a>: and the human environment and thus biologists speak about the evolution of an environment<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3112'>51:52</a>: as well as the evolution of an organism in it. In other words, human beings could not<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3113'>51:53</a>: have appeared on this planet until its temperature had lowered to a certain degree, until the<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3114'>51:54</a>: atmosphere contained certain gases as a result of vegetative development. And then the environment<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=3115'>51:55</a>: became evolved enough for human beings to appear in it. Evolved enough? I'll say if<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=3116'>51:56</a>: something further: intelligent enough for the appearance in it of intelligent beings.</details>

It's more accurate to put our existence into verb language, as distinct from noun language. [[1]](https://youtu.be/NvLlpY9vd9E#t=3121) Nouns have the difficulty of designating things, while verbs designate processes. [[2]](https://youtu.be/NvLlpY9vd9E#t=3122) Everything is really a process. When we speak of "housing" for houses or "matting" for mats, we're getting closer to this idea. [[3]](https://youtu.be/NvLlpY9vd9E#t=3123) The Nootka Indians have a language with no nouns. [[4]](https://youtu.be/NvLlpY9vd9E#t=3124) They say "it houses" and then add an adverb to show whether it houses religiously, homely, or commercially. [[5]](https://youtu.be/NvLlpY9vd9E#t=3125) They see the world as flow. What is the "it" that houses? What is it that rains when we say "it is raining"? [[6]](https://youtu.be/NvLlpY9vd9E#t=3126) [[7]](https://youtu.be/NvLlpY9vd9E#t=3127)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3121'>52:01</a>: Put our existence into verb language, as distinct from noun language, and you're much closer<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3122'>52:02</a>: to the point. You see nouns have the difficulty of designating things. Verbs designate process.<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3123'>52:03</a>: Now everything is a process, really. When we speak of housing for houses, matting for<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3124'>52:04</a>: maps, we're getting there. The Nootka Indians have a language in which there are no nouns.<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3125'>52:05</a>: So they say: it houses. And then they add an adverb to show whether it houses religiously,<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3126'>52:06</a>: homeyly or marketingly. And so they see the world as the flow. What is IT that houses.What<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3127'>52:07</a>: is it that rains when we say it is raining. You see we always have a funny idea that to</details>

We have this odd idea that to get action, you need an agent. [[1]](https://youtu.be/NvLlpY9vd9E#t=3128) But how can a noun start a verb? How can a thing start an event? [[2]](https://youtu.be/NvLlpY9vd9E#t=3129) There's no action in a thing. Action can only come from action, energy from energy. [[3]](https://youtu.be/NvLlpY9vd9E#t=3130) You can't get energy from a concept. Nouns are all concepts, they're abstract. [[4]](https://youtu.be/NvLlpY9vd9E#t=3131) Only verbs are concrete, as the world is process. [[5]](https://youtu.be/NvLlpY9vd9E#t=3132)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3128'>52:08</a>: get a verb, that is to say, to get action you have to have an agent. Now this is the<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3129'>52:09</a>: most ridiculous idea conceivable. How can a noun start a verb? How can a thing start<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3130'>52:10</a>: an event? Because there's no action in a thing. Action can only come from action. Energy from<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3131'>52:11</a>: energy. You can't get energy from a concept. Because nouns are all concepts, they're abstract,<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3132'>52:12</a>: really. It's only verbs that are concrete. As the world is process. Now, common sense</details>

Common sense insists that the pattern of the world must be made of something, because we still think with Aristotle's common sense or the imagery of Genesis where God made Adam out of clay. [[1]](https://youtu.be/NvLlpY9vd9E#t=3133) [[2]](https://youtu.be/NvLlpY9vd9E#t=3134) [[3]](https://youtu.be/NvLlpY9vd9E#t=3135) We constantly think we are made of flesh, as if flesh were some sort of stuff like clay out of which you shape bodies. [[4]](https://youtu.be/NvLlpY9vd9E#t=3136) [[5]](https://youtu.be/NvLlpY9vd9E#t=3137) But are trees made out of wood? What a ridiculous question. Trees are wood. [[6]](https://youtu.be/NvLlpY9vd9E#t=3138) It's this artifact thinking that gives us the idea of the world being made out of something. [[7]](https://youtu.be/NvLlpY9vd9E#t=3139) It isn't made out of anything. [[8]](https://youtu.be/NvLlpY9vd9E#t=3140)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3133'>52:13</a>: insists that the pattern of the world must be made of something. Because we still think<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3134'>52:14</a>: with Aristotle's common sense. Or with the imagery of the Book of Genesis where God made<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3135'>52:15</a>: Adam out of the clay. In other words he made a clay figurine and breathed breath into it<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3136'>52:16</a>: and it became alive. And so we constantly think that we are made of flesh as a flesh<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3137'>52:17</a>: with some sort of stuff like clay out of which you shape bodies or like you make tables out<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3138'>52:18</a>: of wood. Are trees made out of wood? What a ridiculous question. Trees are wood. They're<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3139'>52:19</a>: not made of wood. And it is simply this artifact thing that gives us the idea of the well being<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=3140'>52:20</a>: made out of something. It isn't made out of anything. And so when physics tries to investigate</details>

When physics tries to investigate the "stuff" of matter, it can't find any. [[1]](https://youtu.be/NvLlpY9vd9E#t=3141) You can only talk about process. You can describe what a process is doing, the structure or nature of the dance. [[2]](https://youtu.be/NvLlpY9vd9E#t=3142) [[3]](https://youtu.be/NvLlpY9vd9E#t=3143) [[4]](https://youtu.be/NvLlpY9vd9E#t=3144) But there is nothing doing the performing. There is no "stuff" out of which it's being done because when you examine "stuff" you just find more pattern. [[5]](https://youtu.be/NvLlpY9vd9E#t=3145) What we mean by "stuff" is fuzz. [[6]](https://youtu.be/NvLlpY9vd9E#t=3146) When you look at something out of focus, you see fuzz. But when you come into focus, you see structure. [[7]](https://youtu.be/NvLlpY9vd9E#t=3147) [[8]](https://youtu.be/NvLlpY9vd9E#t=3148) You can see little lines and things, and you want to know what they're made of. [[9]](https://youtu.be/NvLlpY9vd9E#t=3149) So you increase magnification and for a while you get fuzz, but when it's clear again you see those little lines are also made of more little lines and more structure. [[10]](https://youtu.be/NvLlpY9vd9E#t=3150) [[11]](https://youtu.be/NvLlpY9vd9E#t=3151) Big patterns have little patterns upon their backs to bite them, and so on. [[12]](https://youtu.be/NvLlpY9vd9E#t=3152)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3141'>52:21</a>: what is the stuff of matter, it can't find any. Because you can never talk about anything<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3142'>52:22</a>: except a process. You can describe what a process is doing, you can describe the structure,<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3143'>52:23</a>: the nature of the dance, whether it's doing a waltz or a mazurka or the frog or whatever<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3144'>52:24</a>: it is. Then you can describe that melody, shall we say, what it is performing. But there<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3145'>52:25</a>: is nothing doing the performing at all. There is no stuff out of which it's being done because<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3146'>52:26</a>: when you examine stuff you just find more pattern. What you mean by stuff is fuzz. When<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3147'>52:27</a>: you look at something with a lens and you're out of focus you see fuzz. But when you come<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=3148'>52:28</a>: into focus you see structure. Right now the structure is made of all sorts of little lines<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=3149'>52:29</a>: and things. You can see them and you want to what are they made of. So you turn up the<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=3150'>52:30</a>: magnification and for a while you get a lot of fuzz but when you're clear again you see<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=3151'>52:31</a>: that those little lines are also made of more little lines and more structure. Big patterns<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=3152'>52:32</a>: have little patterns upon the backs to bite them, and so on. And that's the way it goes.</details>

Suddenly you might feel insecure because "stuff" has disappeared. [[1]](https://youtu.be/NvLlpY9vd9E#t=3153) There's a story about a physicist who understood this so well that he always went about in enormous padded shoes because he was afraid of falling through the floor. [[2]](https://youtu.be/NvLlpY9vd9E#t=3154) [[3]](https://youtu.be/NvLlpY9vd9E#t=3155)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3153'>52:33</a>: So. Suddenly you feel rather insecure because stuff has disappeared there's a famous story<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3154'>52:34</a>: about a physicist who understood this so well, but he always went about in the most enormous<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3155'>52:35</a>: padded shoes because he was afraid of falling through the floor. Now look what's happened.</details>

If we go through everything I've been saying, we find that what we thought was "I" is nothing more than a social institution, like the equator or an inch. [[1]](https://youtu.be/NvLlpY9vd9E#t=3156) [[2]](https://youtu.be/NvLlpY9vd9E#t=3157) To mistake it, to "reify" it as Whitehead would say, is the fallacy of misplaced concreteness. It's strictly a hallucination. [[3]](https://youtu.be/NvLlpY9vd9E#t=3158) [[4]](https://youtu.be/NvLlpY9vd9E#t=3159) Psychological self-exploration shows this to be the case. [[5]](https://youtu.be/NvLlpY9vd9E#t=3160)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3156'>52:36</a>: Just look and see what has happened to us. If we go through everything that I've been<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3157'>52:37</a>: saying, we find, first of all, that the thing that we thought was I is nothing more than<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3158'>52:38</a>: a social institution. Just like the equator. Or an inch. And to mistake it, to reify it,<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3159'>52:39</a>: as Whitehead would say, is the fallacy of misplaced concreteness. It's a strictly a<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3160'>52:40</a>: hallucination. And any certain amount of psychological self-exploration shows this to be the case.</details>

What we are is the organism, and what the organism is is a transactional interchange between the organism and the environment. [[1]](https://youtu.be/NvLlpY9vd9E#t=3161) It's not quite correct to say you're doing it and it's doing you. [[2]](https://youtu.be/NvLlpY9vd9E#t=3162) Whitehead, in describing scientific philosophy, says that blue is entirely our projection on the sky. He's half right. [[3]](https://youtu.be/NvLlpY9vd9E#t=3163) [[4]](https://youtu.be/NvLlpY9vd9E#t=3164) The part left out of consideration is this: [[5]](https://youtu.be/NvLlpY9vd9E#t=3165) True, you with your optical nerves and eyes transform the sky into the blue feeling. [[6]](https://youtu.be/NvLlpY9vd9E#t=3166) But without the sky, you wouldn't have any optical nerves. It works both ways. [[7]](https://youtu.be/NvLlpY9vd9E#t=3167) Without the air whose density gives the blue effect, there would be no blue. It's mutual. You do it and it does you. But that's a clumsy way of talking about it - it's all one unified process. [[8]](https://youtu.be/NvLlpY9vd9E#t=3168) [[9]](https://youtu.be/NvLlpY9vd9E#t=3169)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3161'>52:41</a>: But what we are is the organism. And what the organism is is a transactional interchange<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3162'>52:42</a>: between the organism and the environment. It's not quite correct to say you're doing<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3163'>52:43</a>: it and it's doing you because, you see, Whitehead, in describing the scientific philosophy, saying,<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3164'>52:44</a>: for example, that blue is entirely our projection on the sky, he's half right, you see. Those<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3165'>52:45</a>: scientists he's talking about are half right. But the part of it that is left out of consideration<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3166'>52:46</a>: is this: True, you with your optical nerves and eyes transform the sky into the blue feeling.<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3167'>52:47</a>: But without the sky you wouldn't have any optical nerves. It works both ways you see.<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=3168'>52:48</a>: Without the air whose density gives the blue effect. It's mutual. You do it and it does<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=3169'>52:49</a>: you. But that's, as I said, a two-way, a clumsy way of talking about: it's all one process,</details>

Furthermore, from this process has disappeared what we thought was solidity, what we thought was common sense substance and stuff - it's just pattern. [[1]](https://youtu.be/NvLlpY9vd9E#t=3170) At once one feels sort of ghostly, as if you could be easily blown away. [[2]](https://youtu.be/NvLlpY9vd9E#t=3171) That's why the Hindus call the universe Maya, which means illusion. [[3]](https://youtu.be/NvLlpY9vd9E#t=3172) Don't forget "illusion" is related to the Latin word "ludere" - to play. [[4]](https://youtu.be/NvLlpY9vd9E#t=3173) So it's the play, the big act. It also means magic, as in a conjuror's creation of an illusion. [[5]](https://youtu.be/NvLlpY9vd9E#t=3174) It also means creative power, art, and finally the divine power - the Maya of the Lord. [[6]](https://youtu.be/NvLlpY9vd9E#t=3175) [[7]](https://youtu.be/NvLlpY9vd9E#t=3176)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3170'>52:50</a>: a unified process. And furthermore from this process there has disappeared what we thought<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3171'>52:51</a>: was solidity. What we thought was common sense, substance and stuff- it's just pattern. And<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3172'>52:52</a>: at once one feels sort of ghostly. As if you could be easily blown away. And that's why<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3173'>52:53</a>: the Hindus call the universe the Maya. Which means the illusion. Don't forget illusion<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3174'>52:54</a>: is related to the Latin word ludere- to play. So the play. Big act. It also means magic.<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3175'>52:55</a>: As in a conjuror's creation of an illusion. It also means creative power. It also means<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3176'>52:56</a>: art. Finally it means the divine power. The Maya of the Lord. Lord is a bad translation</details>

Having arranged this general introduction, which may be familiar to some of you who've attended my seminars before, I've raised the question: [[1]](https://youtu.be/NvLlpY9vd9E#t=3177) [[2]](https://youtu.be/NvLlpY9vd9E#t=3178) If that's the way it is, how on earth are we going to arrange a transformation of human consciousness so that we'll know it? [[3]](https://youtu.be/NvLlpY9vd9E#t=3179) Not just in theory, but something we feel in the same way as you feel what you take to be "I" at the present moment, confronting an external world. [[4]](https://youtu.be/NvLlpY9vd9E#t=3180) [[5]](https://youtu.be/NvLlpY9vd9E#t=3181) How will you transform that sensation? [[6]](https://youtu.be/NvLlpY9vd9E#t=3182) Because if you don't transform that sensation, you will never be fit to use technology. [[7]](https://youtu.be/NvLlpY9vd9E#t=3183) We shall continue to use our technology in a hostile spirit towards the external world and we shall wreck it. [[8]](https://youtu.be/NvLlpY9vd9E#t=3184) We're busy doing it now. [[9]](https://youtu.be/NvLlpY9vd9E#t=3185) There is no necessity to abandon technology. We can adapt. [[10]](https://youtu.be/NvLlpY9vd9E#t=3186) But we can certainly use it in a different spirit. [[11]](https://youtu.be/NvLlpY9vd9E#t=3187)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3177'>52:57</a>: of Bhagavan. Just the Divine One. Lord means boss and the Hindus don't do it that way.<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3178'>52:58</a>: But now you see, having arranged this general introduction, which I'm afraid will be familiar<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3179'>52:59</a>: to some of you who've attended my seminars before; but I've arranged this general introduction<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3180'>53:00</a>: to raise the question: All right. If that's the way it is. How on earth are we going to<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3181'>53:01</a>: arrange a transformation of man's consciousness so that he'll know it. Not just in theory<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3182'>53:02</a>: but something he feels in the same way as you feel what you take to be I at the present<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3183'>53:03</a>: moment, confronting an external world. How will you transform that sensation? Because<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=3184'>53:04</a>: if you don't transform that sensation, you are not ever be fit to use technology. We<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=3185'>53:05</a>: shall continue to use our technology in a hostile spirit towards the external world<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=3186'>53:06</a>: and we shall wreck the external world. We're busy doing it now. There is no necessity to<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=3187'>53:07</a>: abandon technology. We can't adapt. But we can certainly use it in a different spirit.</details>

I've just been in Ceylon, which is a beautiful garden but completely undeveloped from a technical point of view and in very bad economic circumstances. [[1]](https://youtu.be/NvLlpY9vd9E#t=3188) [[2]](https://youtu.be/NvLlpY9vd9E#t=3189) [[3]](https://youtu.be/NvLlpY9vd9E#t=3190) The change has got to come. [[4]](https://youtu.be/NvLlpY9vd9E#t=3191) I discussed with a high member of the government the possibility of setting up an experimental station in Ceylon - an institute of ecological technology. [[5]](https://youtu.be/NvLlpY9vd9E#t=3192) [[6]](https://youtu.be/NvLlpY9vd9E#t=3193) We could work out ways of production, mechanization, and automation that would not ruin the island. [[7]](https://youtu.be/NvLlpY9vd9E#t=3194) [[8]](https://youtu.be/NvLlpY9vd9E#t=3195) You have to do that sort of thing with dedication because one reason we make such a mess with technology is that shareholders in corporations want to make a fast buck. [[9]](https://youtu.be/NvLlpY9vd9E#t=3196) [[10]](https://youtu.be/NvLlpY9vd9E#t=3197)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3188'>53:08</a>: I've just been in the island of Ceylon, which is a garden. A beautiful beautiful place.<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3189'>53:09</a>: But it's completely undeveloped from a technical point of view and it's in very bad economic<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3190'>53:10</a>: circumstances because nobody wants to buy natural rubber anymore. It has no foreign<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3191'>53:11</a>: exchange. It is very peaceful. But the change has got to come. So I discussed with one of<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3192'>53:12</a>: the high members of the government the possibility that we could set up in Ceylon an experimental<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3193'>53:13</a>: station. Which would serve not only India eventually and Africa, but us, too. A an institute<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=3194'>53:14</a>: of ecological technology. Where we could in that experimental Island work out ways of<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=3195'>53:15</a>: production, of mechanization, automation and so on which would not ruin the island. And<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=3196'>53:16</a>: you have to do that sort of thing with a certain dedication because one of the reasons why<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=3197'>53:17</a>: we make such a mess with technology. is that the shareholders in any given corporation</details>

This is another example of confusing the symbol with the reality, what [[1]](https://youtu.be/NvLlpY9vd9E#t=3207) Alfred North Whitehead calls the "fallacy of misplaced concreteness." [[2]](https://youtu.be/NvLlpY9vd9E#t=3208) We should devote the rest of [[3]](https://youtu.be/NvLlpY9vd9E#t=3209) our discussion to exploring the various ways in which it is proposed that we bring about [[4]](https://youtu.be/NvLlpY9vd9E#t=3210) or assist in bringing about that change in our perception and conception of our own [[5]](https://youtu.be/NvLlpY9vd9E#t=3211) existence. This will allow us to feel ourselves as we truly are, as distinct from the way we've been conditioned to perceive ourselves. [[6]](https://youtu.be/NvLlpY9vd9E#t=3212)
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=3207'>53:27</a>: again, you see, this is another example of confusing the symbol with the reality, the<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=3208'>53:28</a>: fallacy of misplaced concreteness as Whitehead calls it. So we should devote the rest of<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=3209'>53:29</a>: the seminar to discussing the various ways in which it is proposed that we bring about<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=3210'>53:30</a>: or assist the bringing about of that change in our perception and conception of our own<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=3211'>53:31</a>: existence. So that we can feel ourselves the way we are. As distinct from the way in which<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=3212'>53:32</a>: we've been told to feel ourselves.</details>


# Prompt your content

Use your content to create context and prompt it:
* Set `PROMPT`
* Set `CONTEXT_SIZE`:
- `"small"` - only paragraphs will be used.
- `"medium"` - only transcript will be used.
- `"large"` - both paragraphs and transcript will be used.

In [10]:
#@title Prompt your context


PROMPT = "Suggest an extended opening line to a conversation about this <context>."
CONTEXT_SIZE = "small"

response = llm.prompt(
    prompt=PROMPT,
    context=[content],
    context_size=CONTEXT_SIZE,
)
display(Markdown(render_paragraph(response)))

"Have you ever considered that our common understanding of 'self' might be an illusion, and that we're actually part of a vast, interconnected process with the universe? This fascinating perspective, explored by thinkers like Alfred North Whitehead and echoed in Hindu philosophy, challenges our traditional notions of individuality and separateness [[1]](https://youtu.be/NvLlpY9vd9E#t=97) [[2]](https://youtu.be/NvLlpY9vd9E#t=106) [[3]](https://youtu.be/NvLlpY9vd9E#t=114). It suggests that rather than being isolated entities, we're more like whirlpools in the stream of existence [[97]](https://youtu.be/NvLlpY9vd9E#t=855) [[98]](https://youtu.be/NvLlpY9vd9E#t=871) [[99]](https://youtu.be/NvLlpY9vd9E#t=877) [[100]](https://youtu.be/NvLlpY9vd9E#t=884), constantly changing yet maintaining a recognizable form. This radical shift in perception could have profound implications for how we approach technology, environmental issues, and our relationship with nature [[45]](https://youtu.be/NvLlpY9vd9E#t=441) [[46]](https://youtu.be/NvLlpY9vd9E#t=449) [[47]](https://youtu.be/NvLlpY9vd9E#t=455) [[48]](https://youtu.be/NvLlpY9vd9E#t=464) [[49]](https://youtu.be/NvLlpY9vd9E#t=471). Shall we delve into this mind-bending concept and explore its potential to transform human consciousness and our approach to the world around us?"
<details><summary>Transcript</summary>1. <a href='https://youtu.be/NvLlpY9vd9E#t=97'>01:37</a>: We talk in ordinary ways about my body. My feet. And when we go to the dentist to have<br>2. <a href='https://youtu.be/NvLlpY9vd9E#t=106'>01:46</a>: out teeth fixed, we regard him rather as a mechanic. Like you take your car to the garage<br>3. <a href='https://youtu.be/NvLlpY9vd9E#t=114'>01:54</a>: so you take your body to the surgeon or the dentist or whatever it is to be fixed how<br>4. <a href='https://youtu.be/NvLlpY9vd9E#t=118'>01:58</a>: the parts changed or something of that kind. And they're really getting to work on that<br>5. <a href='https://youtu.be/NvLlpY9vd9E#t=124'>02:04</a>: now. And so the question is when somebody has a heart transplant. That sounds very radical<br>6. <a href='https://youtu.be/NvLlpY9vd9E#t=132'>02:12</a>: because we say in my heart of hearts. But nowadays most of us seem to feel that whatever<br>7. <a href='https://youtu.be/NvLlpY9vd9E#t=141'>02:21</a>: it is, that I is, is located in the head. Somewhere behind the eyes and between the<br>8. <a href='https://youtu.be/NvLlpY9vd9E#t=150'>02:30</a>: ears is the center. And the rest of us is an appendage a vehicle which carries the self<br>9. <a href='https://youtu.be/NvLlpY9vd9E#t=157'>02:37</a>: around. Now popular speech also reflects the sensation. That I am very different from what<br>10. <a href='https://youtu.be/NvLlpY9vd9E#t=171'>02:51</a>: we call the other. Other people, other things. Anything that we can become aware of is sort<br>11. <a href='https://youtu.be/NvLlpY9vd9E#t=179'>02:59</a>: of other. There is an opposition apparently between the knower and the known. And so we<br>12. <a href='https://youtu.be/NvLlpY9vd9E#t=186'>03:06</a>: talk about facing reality. We talk about coming into this world. As if somehow we didn't belong.<br>13. <a href='https://youtu.be/NvLlpY9vd9E#t=196'>03:16</a>: As if instead of being leaves growing out of a tree we were a lot of birds that had<br>14. <a href='https://youtu.be/NvLlpY9vd9E#t=201'>03:21</a>: alighted on bare branches. And it has become common sense for most people living in the<br>15. <a href='https://youtu.be/NvLlpY9vd9E#t=210'>03:30</a>: twentieth century today to adopt the nineteenth century philosophy of science. Which interprets<br>16. <a href='https://youtu.be/NvLlpY9vd9E#t=220'>03:40</a>: the physical universe outside human bodies as being a mechanical contraption which is<br>17. <a href='https://youtu.be/NvLlpY9vd9E#t=229'>03:49</a>: essentially stupid, unfeeling automatic, Composed of mainly geological elements: rocks gases<br>18. <a href='https://youtu.be/NvLlpY9vd9E#t=240'>04:00</a>: and so forth. And therefore we feel rather alone and left out of this thing. In contrast<br>19. <a href='https://youtu.be/NvLlpY9vd9E#t=248'>04:08</a>: with the ideas of Ptolemaic astronomy. Instead of being at the center of the universe. We<br>20. <a href='https://youtu.be/NvLlpY9vd9E#t=257'>04:17</a>: are on the outer limits of a minor galaxy revolving around an unimportant star. On a<br>21. <a href='https://youtu.be/NvLlpY9vd9E#t=265'>04:25</a>: small minute ball of rock. And therefore that astronomical way of looking at things is simply<br>22. <a href='https://youtu.be/NvLlpY9vd9E#t=275'>04:35</a>: overwhelming. It makes us feel not only of no importance but also very much left out.<br>23. <a href='https://youtu.be/NvLlpY9vd9E#t=284'>04:44</a>: And as a result, that is the common sense of most people living today.<br>24. <a href='https://youtu.be/NvLlpY9vd9E#t=290'>04:50</a>: We did of course have a religious view of our nature that we were the children of a<br>25. <a href='https://youtu.be/NvLlpY9vd9E#t=294'>04:54</a>: loving God who is in charge of this whole operation. But very few people actually believe<br>26. <a href='https://youtu.be/NvLlpY9vd9E#t=301'>05:01</a>: that anymore. A great many people think they ought to believe it and would like to believe<br>27. <a href='https://youtu.be/NvLlpY9vd9E#t=306'>05:06</a>: in it but they don't. Most ministers that I know don't believe it but they feel guilty<br>28. <a href='https://youtu.be/NvLlpY9vd9E#t=311'>05:11</a>: about this because they feel they ought to. But it became implausible. There never was<br>29. <a href='https://youtu.be/NvLlpY9vd9E#t=317'>05:17</a>: a serious argument against it. It simply became unthinkable in comparison with the dimensions<br>30. <a href='https://youtu.be/NvLlpY9vd9E#t=323'>05:23</a>: of the universe as we now see it. So having lost a way of looking at the world, an image<br>31. <a href='https://youtu.be/NvLlpY9vd9E#t=332'>05:32</a>: of the world which gave us some sense of meaning, we now have an image of the world which gives<br>32. <a href='https://youtu.be/NvLlpY9vd9E#t=337'>05:37</a>: us none at all. And so we feel rather inclined to put up a fight against the whole show.<br>33. <a href='https://youtu.be/NvLlpY9vd9E#t=346'>05:46</a>: Interestingly enough, when in the nineteenth century we switched our common sense from<br>34. <a href='https://youtu.be/NvLlpY9vd9E#t=352'>05:52</a>: supernaturalism to naturalism, one would think that a naturalist would be a person who love<br>35. <a href='https://youtu.be/NvLlpY9vd9E#t=360'>06:00</a>: nature. Just as a materialist ought to be a person who loves material but certainly<br>36. <a href='https://youtu.be/NvLlpY9vd9E#t=364'>06:04</a>: isn't. With what is called a philosophy of scientific naturalism. Naturalism is used<br>37. <a href='https://youtu.be/NvLlpY9vd9E#t=374'>06:14</a>: in a negative way. It has nothing to do with being natural it has something to do with<br>38. <a href='https://youtu.be/NvLlpY9vd9E#t=380'>06:20</a>: being not supernatural, Merely natural. And all sorts of phrases with coined in that epoch<br>39. <a href='https://youtu.be/NvLlpY9vd9E#t=387'>06:27</a>: which I would call put down phrases. Freud spoke of the basic psychic energy as Libido,<br>40. <a href='https://youtu.be/NvLlpY9vd9E#t=396'>06:36</a>: which means blind lust. People like Ernst Hegel spoke of the universe as being a manifestation<br>41. <a href='https://youtu.be/NvLlpY9vd9E#t=404'>06:44</a>: of blind energy. Think of that put down word blind. And therefore we also speak of unconscious<br>42. <a href='https://youtu.be/NvLlpY9vd9E#t=413'>06:53</a>: mental mechanisms. And the very word unconscious as being the deeper aspect of our psyche is<br>43. <a href='https://youtu.be/NvLlpY9vd9E#t=421'>07:01</a>: a negative word and a put down word. So is to say what you are functioning as a rational<br>44. <a href='https://youtu.be/NvLlpY9vd9E#t=429'>07:09</a>: ego with values and with a capacity to love, is simply the epiphenomenon of a purely mechanical<br>45. <a href='https://youtu.be/NvLlpY9vd9E#t=441'>07:21</a>: process. To bad. So as a result of this so-called naturalism we began to put up the most whopping<br>46. <a href='https://youtu.be/NvLlpY9vd9E#t=449'>07:29</a>: fight against nature that was ever engaged in. And that fight is an expression of our<br>47. <a href='https://youtu.be/NvLlpY9vd9E#t=455'>07:35</a>: fury and of a feeling of being left out. So that the technological experiment which became<br>48. <a href='https://youtu.be/NvLlpY9vd9E#t=464'>07:44</a>: possible as a result of the mechanical Sciences has largely been conducted in a spirit of<br>49. <a href='https://youtu.be/NvLlpY9vd9E#t=471'>07:51</a>: rage. And the results are evident all around us. Here in Palm Springs. You are gradually<br>50. <a href='https://youtu.be/NvLlpY9vd9E#t=481'>08:01</a>: getting all the smog from Los Angeles. This great cloud of poisonous gas put up by a city<br>51. <a href='https://youtu.be/NvLlpY9vd9E#t=489'>08:09</a>: which is exemplary in this whole civilized world for fouling it's own nest. Perhaps only<br>52. <a href='https://youtu.be/NvLlpY9vd9E#t=499'>08:19</a>: Calcutta could be a bit worse. Or some such terrible slum. But we have done it by technology.<br>53. <a href='https://youtu.be/NvLlpY9vd9E#t=510'>08:30</a>: By ruthless beating about of nature without consideration for what the scientist would<br>54. <a href='https://youtu.be/NvLlpY9vd9E#t=520'>08:40</a>: call our ecology.<br>55. <a href='https://youtu.be/NvLlpY9vd9E#t=523'>08:43</a>: Ecology is that aspect of science which deals with the relationship between organisms and<br>56. <a href='https://youtu.be/NvLlpY9vd9E#t=529'>08:49</a>: their environments. Ecology is the study of the balance of nature. Of the way in which<br>57. <a href='https://youtu.be/NvLlpY9vd9E#t=536'>08:56</a>: every living being depends upon innumerable other living beings of all species. And also<br>58. <a href='https://youtu.be/NvLlpY9vd9E#t=545'>09:05</a>: upon inanimate forces. Air, water, temperature, gases, vegetation and all sorts of things.<br>59. <a href='https://youtu.be/NvLlpY9vd9E#t=555'>09:15</a>: And this is one of the most important sciences that we can possibly study today. Because<br>60. <a href='https://youtu.be/NvLlpY9vd9E#t=561'>09:21</a>: we are in a position where we realize that we cannot help interfering with the world.<br>61. <a href='https://youtu.be/NvLlpY9vd9E#t=567'>09:27</a>: To be alive is to interfere. You must interfere. You cannot go back and say: Hands off nature.<br>62. <a href='https://youtu.be/NvLlpY9vd9E#t=574'>09:34</a>: let's leave it all alone. Because you're stuck with it. Especially once you've started to<br>63. <a href='https://youtu.be/NvLlpY9vd9E#t=580'>09:40</a>: interfere in a major way. We have so altered our environment that there is no hope for<br>64. <a href='https://youtu.be/NvLlpY9vd9E#t=586'>09:46</a>: it but to go ahead. But we can, to some extent, change direction. But the only way that I<br>65. <a href='https://youtu.be/NvLlpY9vd9E#t=593'>09:53</a>: can see of our effectively changing direction is through a transformation of the feeling<br>66. <a href='https://youtu.be/NvLlpY9vd9E#t=603'>10:03</a>: that we have of our own existence and of what we mean by I. The reason for this is simply<br>67. <a href='https://youtu.be/NvLlpY9vd9E#t=612'>10:12</a>: that all kinds of intelligent and even powerful people like, say, Laurance Rockefeller who<br>68. <a href='https://youtu.be/NvLlpY9vd9E#t=618'>10:18</a>: are interested in ecology and in conservation of our natural resources, they can scream<br>69. <a href='https://youtu.be/NvLlpY9vd9E#t=624'>10:24</a>: their heads off but nobody pays any attention. There is as yet no really serious program<br>70. <a href='https://youtu.be/NvLlpY9vd9E#t=632'>10:32</a>: at the government level to do anything radical about the pollution of water, the waste of<br>71. <a href='https://youtu.be/NvLlpY9vd9E#t=639'>10:39</a>: water, pollution of air and the general ravaging of the United States of America. I'm amazed<br>72. <a href='https://youtu.be/NvLlpY9vd9E#t=648'>10:48</a>: that congressman can pass a bill imposing severe penalties on anyone who burns the American<br>73. <a href='https://youtu.be/NvLlpY9vd9E#t=654'>10:54</a>: flag, whereas they are responsible for burning that for which the flag stands. The United<br>74. <a href='https://youtu.be/NvLlpY9vd9E#t=660'>11:00</a>: States as a territory, as a people, and as a biological manifestation. That is an example<br>75. <a href='https://youtu.be/NvLlpY9vd9E#t=670'>11:10</a>: of our perennial confusion of symbols with realities. Which is in a way at the heart<br>76. <a href='https://youtu.be/NvLlpY9vd9E#t=678'>11:18</a>: of the trouble because what we think of as I is much more a symbol than it is a reality.<br>77. <a href='https://youtu.be/NvLlpY9vd9E#t=687'>11:27</a>: The living organism, the whole mind-body, is much more than anything we mean by I. I<br>78. <a href='https://youtu.be/NvLlpY9vd9E#t=696'>11:36</a>: largely stands for your personality. Your role in life. And the very word person, as<br>79. <a href='https://youtu.be/NvLlpY9vd9E#t=705'>11:45</a>: you probably know, comes from the Latin persona. A word originally used for the mask worn by<br>80. <a href='https://youtu.be/NvLlpY9vd9E#t=713'>11:53</a>: actors in Graeco-Roman drama. That through which sound comes, because the mask had a<br>81. <a href='https://youtu.be/NvLlpY9vd9E#t=719'>11:59</a>: mega phonic mouth to carry the voice an open air theaters. So when you speak of being a<br>82. <a href='https://youtu.be/NvLlpY9vd9E#t=726'>12:06</a>: real person, it really means being a genuine fake. Because the personality is only the<br>83. <a href='https://youtu.be/NvLlpY9vd9E#t=734'>12:14</a>: front. What is behind it? Well of course the organism is behind it. The whole organism.<br>84. <a href='https://youtu.be/NvLlpY9vd9E#t=744'>12:24</a>: And we must be very careful not to confuse the organism with various symbols that we<br>85. <a href='https://youtu.be/NvLlpY9vd9E#t=751'>12:31</a>: have for it, because those symbols can be extremely misleading. If we say the organism<br>86. <a href='https://youtu.be/NvLlpY9vd9E#t=757'>12:37</a>: is the body, what we usually mean by the body, is an impoverished meaning. When we speak<br>87. <a href='https://youtu.be/NvLlpY9vd9E#t=767'>12:47</a>: of my body that is to say my vehicle, my physical automobile. That is an unenriched meaning<br>88. <a href='https://youtu.be/NvLlpY9vd9E#t=777'>12:57</a>: of the word body. Because what you really are as a body, as a living organism, is not<br>89. <a href='https://youtu.be/NvLlpY9vd9E#t=787'>13:07</a>: some sort of separate existence coated by a skin which divides you from the rest of<br>90. <a href='https://youtu.be/NvLlpY9vd9E#t=795'>13:15</a>: the world. Shakespeare has King John saying to Hubert: "Within this wall of flesh there<br>91. <a href='https://youtu.be/NvLlpY9vd9E#t=802'>13:22</a>: is a soul counts thee her creditor." Within this wall of flesh: the skin considered as<br>92. <a href='https://youtu.be/NvLlpY9vd9E#t=809'>13:29</a>: a barrier. When actually, from a biological point of view, the human skin and all skins<br>93. <a href='https://youtu.be/NvLlpY9vd9E#t=816'>13:36</a>: are osmotic membranes. You know when you get something by osmosis. By sort of soaking it<br>94. <a href='https://youtu.be/NvLlpY9vd9E#t=824'>13:44</a>: in. So in the same way one's skin is a spongy construction full of holes. Full of communicators,<br>95. <a href='https://youtu.be/NvLlpY9vd9E#t=832'>13:52</a>: nerve endings. And your skin is simply a vibrating membrane through which the so-called external<br>96. <a href='https://youtu.be/NvLlpY9vd9E#t=842'>14:02</a>: world flows into you and through you. So that you yourself actually are not so much an entity<br>97. <a href='https://youtu.be/NvLlpY9vd9E#t=855'>14:15</a>: that moves around in an environment. You are much more like a whirlpool in a stream. And,<br>98. <a href='https://youtu.be/NvLlpY9vd9E#t=871'>14:31</a>: as you know, the whirlpool is constant only in its doing, that is to say in its whirling.<br>99. <a href='https://youtu.be/NvLlpY9vd9E#t=877'>14:37</a>: And you could recognize individual whirligigs in a stream. But the water is flowing through<br>100. <a href='https://youtu.be/NvLlpY9vd9E#t=884'>14:44</a>: them all the time. They are never the same but for a second. And so it is also with us.</details>